In [112]:
# Identify athletes meeting OCTC selection rules

#1. At least 1 athlete per event per gender
#2. Except for 100m, 400m, cap at 3 athletes per event. 100m, 400m capped at 6
#3. Where top athlete is >30 yrs old (except marathon), to include next athlete as well (below 30)
#4. Where althlete qualified in 2 events, to choose the better performing one
#5. For athletes looking to do full time, to write in to SAA for special consideration
#6. Exclude SPEX carded athletes
#7. Except for marathon, age threshold cut off of 40 yrs old for top athlete
#8. No double tapping of prog - potential names in red


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1537]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic
import dateutil.parser as parser 


from google.cloud import storage



In [1538]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1539]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,"Koh, Qi En Evie",6.41m,Singapore Sports School,13,11,U15,Shot Put,0,(3kg),nan,4/6/12,nan,None,Throw,Female,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
1,"Raihan Shah, Marsya Raudhah Shah",5.98m,Singapore Sports School,14,13,U15,Shot Put,0,(3kg),nan,30/1/11,nan,None,Throw,Female,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
2,"Zhou, Yuhan",6.21m,Singapore Sports School,14,12,U15,Shot Put,0,(3kg),nan,28/9/11,nan,None,Throw,Female,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
3,"TAN, JOSE HONG RUI",12.88m,VICTORIA SCHOOL,15,5,U18,Shot Put,0,(5kg),J656E10,5/2/10,SIN,None,Throw,Male,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
4,"LOO, HAOYUE",9.21m,VICTORIA SCHOOL,15,11,U18,Shot Put,0,(5kg),H232J10,25/7/10,SIN,None,Throw,Male,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,Kampton Kam,2.15,None,None,1,None,High Jump,None,None,None,6-Mar-01,SGP,None,Jump,Male,Penn Classic,8-Feb,2025,International
26005,Marc Brian Louis,6.79,None,None,5,None,60m,None,None,None,7-Aug-02,SGP,None,Sprint,Male,The Keely Klassic,15-Feb,2025,International
26006,Kampton Kam,2.16,None,None,3,None,High Jump,None,None,None,6-Mar-01,SGP,None,Jump,Male,Tiger Paw Invitational,14 - 15 February,2025,International
26007,Joshua Chua Hanwei,11.16,None,None,4,None,100m,None,None,None,20-Jan-00,SGP,0.1,Sprint,Male,Victorian T&F Champs,21 - 23 February,2025,International


In [1540]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


competitors.to_csv('athletes_new_format_prod.csv', sep=',', encoding='utf-8-sig', index=False)

In [1541]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,"Koh, Qi En Evie",6.41m,Singapore Sports School,13,11,U15,Shot Put,0,(3kg),nan,4/6/12,nan,None,Throw,Female,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
1,"Raihan Shah, Marsya Raudhah Shah",5.98m,Singapore Sports School,14,13,U15,Shot Put,0,(3kg),nan,30/1/11,nan,None,Throw,Female,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
2,"Zhou, Yuhan",6.21m,Singapore Sports School,14,12,U15,Shot Put,0,(3kg),nan,28/9/11,nan,None,Throw,Female,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
3,"TAN, JOSE HONG RUI",12.88m,VICTORIA SCHOOL,15,5,U18,Shot Put,0,(5kg),J656E10,5/2/10,SIN,None,Throw,Male,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
4,"LOO, HAOYUE",9.21m,VICTORIA SCHOOL,15,11,U18,Shot Put,0,(5kg),H232J10,25/7/10,SIN,None,Throw,Male,SA Age Group Championships 2025,3/13 to 3/30,2025,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,Kampton Kam,2.15,None,None,1,None,High Jump,None,None,None,6-Mar-01,SGP,None,Jump,Male,Penn Classic,8-Feb,2025,International
26005,Marc Brian Louis,6.79,None,None,5,None,60m,None,None,None,7-Aug-02,SGP,None,Sprint,Male,The Keely Klassic,15-Feb,2025,International
26006,Kampton Kam,2.16,None,None,3,None,High Jump,None,None,None,6-Mar-01,SGP,None,Jump,Male,Tiger Paw Invitational,14 - 15 February,2025,International
26007,Joshua Chua Hanwei,11.16,None,None,4,None,100m,None,None,None,20-Jan-00,SGP,0.1,Sprint,Male,Victorian T&F Champs,21 - 23 February,2025,International


In [1542]:
competitors[competitors['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
5737,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,1991,SGP,-,Mid,Male,Vitaly West Mininster Mile 5K (UK),22-Sep,2024,International
5739,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,1991,SGP,-,Long,Male,"Tokai University Long Distance Challenge 10,0...",25-Nov,2024,International
5740,SOH RUI YONG GUILLAUME,15:07.7,INDIVIDUAL,33,46,None,5000m,None,None,None,1991,SGP,-,Long,Male,Friday Night Under the Lights 5K (UK),13-Sep,2024,International
5741,SOH RUI YONG GUILLAUME,08:35.8,INDIVIDUAL,33,None,None,3000m,None,None,None,1991,SGP,-,Long,Male,Hercules Wimbledon Athletics Club 3000m,14-Jun,2024,International


In [1543]:
#athletes_selected = competitors[(competitors['YEAR']=='2024')|(competitors['YEAR']=='2025')]

athletes_selected = competitors[(competitors['YEAR']=='2024')]

In [1544]:
athletes_selected

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
5079,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
5080,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
5081,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local
5082,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
5083,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23941,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local
23942,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,nan,nan,,Relay,Female,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local
23943,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local
23944,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,nan,nan,,Relay,Female,11th Club ZOOM Kindred Spirit 2024 & PEERS,11/23 to 11/24,2024,Local


In [1545]:
athletes_selected[athletes_selected['NAME']=='Caleb Hia']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
6576,Caleb Hia,2:30:15,None,None,193,None,Marathon,None,None,None,None,SGP,None,Marathon,Male,TCS London Marathon,21-Apr,2024,International


In [1546]:
athletes_selected.to_csv('athletes_selected_prod.csv', encoding='utf-8')

In [1547]:
# Choose 2024/25 only

athletes = athletes_selected

In [1548]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
5079,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
5080,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
5081,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local
5082,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
5083,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23941,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local
23942,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,nan,nan,,Relay,Female,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local
23943,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local
23944,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,nan,nan,,Relay,Female,11th Club ZOOM Kindred Spirit 2024 & PEERS,11/23 to 11/24,2024,Local


In [1549]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
6492,Jun Jie Calvin Quek,53.19,None,None,4,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,UAE Athletics Grand Prix,3-May,2024,International
6493,Jun Jie Calvin Quek,52.05,None,None,8,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,11th Kinami Michitaka Memorial Athletics Meet,12-May,2024,International
6595,Jun Jie Calvin Quek,51.8,None,None,1,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,Taiwan Athletics Open,1 - 2 June,2024,International
6623,Jun Jie Calvin Quek,51.79,None,None,2,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,XXXIII Qosanov Memorial,22 - 23 June,2024,International
6624,Jun Jie Calvin Quek,56.95,None,None,3,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,XXXIII Qosanov Memorial,22 - 23 June,2024,International
6768,Jun Jie Calvin Quek,52.18,None,None,3,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,27 - 29 May,2024,International
6769,Jun Jie Calvin Quek,51.33,None,None,2,None,400m Hurdles,None,None,None,26-Feb-96,SGP,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,27 - 29 May,2024,International


In [1550]:
# Run events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m')

# Hurdles events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110 Meter Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110m Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Hurdles.+', value='100m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Hurdles.+', value='200m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Hurdles.+', value='400m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400m Hurdles.+', value='400m hurdles')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Race Walk.+', value='1500m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000m Race Walk.+', value='3000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000m Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Race Walk.+', value='10000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Race Walk.+', value='race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Shot Put.+', value='Shot put')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Discus.+', value='Discus throw')

#mask = athletes['EVENT'].str.contains(r'Discus', na=True)
#athletes.loc[mask, 'EVENT'] = 'Discus throw'

#mask = athletes['EVENT'].str.contains(r'Shot', na=True)
#athletes.loc[mask, 'EVENT'] = 'Shot put'

#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'EVENT'] = 'Javelin throw'



#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw.+', value='Javelin throw')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump.+', value='Long jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump.+', value='High jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase.+', value='3000m steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+2000 Meter Steeplechase.+', value='2000m steeplechase')


#mask = athletes['EVENT'].str.contains(r'High', na=True)
#athletes.loc[mask, 'EVENT'] = 'High jump'

#mask = athletes['EVENT'].str.contains(r'110m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '110m hurdles'

#mask = athletes['EVENT'].str.contains(r'400m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '400m hurdles'

#mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '200m hurdles'

#mask = athletes['EVENT'].str.contains(r'100m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '100m hurdles'

#mask = athletes['EVENT'].str.contains(r'4 X 100m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4 X 400m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'

#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'Hammer Throw', na=True)
#athletes.loc[mask, 'EVENT'] = 'Hammer throw'

#mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '3000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'


#mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'


# correct javelin category
#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'




In [1551]:
# Wind mapping

#mask = athletes['EVENT'].str.contains(r'\d{1}.\d{1}', na=True, regex=True)
#athletes.loc[mask, 'WIND_MAP'] = '400m'


In [1552]:
# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'50', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'60', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'80', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'^600$', na=True, regex=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10\,000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'^80m\sHurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = athletes['EVENT'].str.contains(r'^80m\shurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = athletes['EVENT'].str.contains(r'^80\sMeter\sHurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'80', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'



mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = ((athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False)) 
         & ((athletes['EVENT_CLASS'].str.contains('None', na=False))|(athletes['EVENT_CLASS']==np.nan)|(athletes['EVENT_CLASS']=='')) 
         & athletes['REGION'].str.contains(r'International', na=False) & (athletes['DIVISION'].str.contains(r'None', na=False)))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open|Invitational', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('None|0.762|0.914', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'



# Throws


#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'600g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'800g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='4kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7.26', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'7.26kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'4.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=False))  & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


#mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=False)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'2kg|2.00kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1061855655.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['MAPPED_EVENT']=''


In [1553]:
athletes[(athletes['EVENT']=='110m Hurdles') & (athletes['EVENT_CLASS']=='None')]

#athletes[(athletes['EVENT']=='110m Hurdles')].tail(50)

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,MAPPED_EVENT


In [1554]:
athletes[athletes['MAPPED_EVENT']=='Marathon']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,MAPPED_EVENT
5627,"HE, HENRY YONG",2:44:28,Singapore,41,20,None,Marathon,None,None,None,1983,SGP,-,Marathon,Male,Asian Marathon,21-Jan,2024,International,Marathon
5744,DEVATHAS SATIANATHAN?,2:56:03,INDIVIDUAL,-,None,None,Marathon,None,None,None,-,SGP,-,Marathon,Male,Boston Marathon (USA),15-Apr,2024,International,Marathon
5745,"HIA, CALEB",2:43:40,INDIVIDUAL,-,190,None,Marathon,None,None,None,-,SGP,-,Marathon,Male,Chicago Marathon (USA),13-Oct,2024,International,Marathon
5747,"TAN, YONG JIN, KERWIN",2:49:39,INDIVIDUAL,-,None,None,Marathon,None,None,None,-,SGP,-,Marathon,Male,Chicago Marathon (USA),13-Oct,2024,International,Marathon
5748,ANSGAR CHENG,2:51:45,INDIVIDUAL,58,None,None,Marathon,None,None,None,1966,SGP,-,Marathon,Male,Berlin Marathon (Germany),29-Sep,2024,International,Marathon
5749,PU YE,3:22:38,INDIVIDUAL,54,None,None,Marathon,None,None,None,1970,SGP,-,Marathon,Male,Tokyo Marathon (Japan),3-Mar,2024,International,Marathon
5750,ALAN YUMING CAO,2:50:35,INDIVIDUAL,48,None,None,Marathon,None,None,None,1976,SGP,-,Marathon,Male,Wuxi Marathon (China),24-Mar,2024,International,Marathon
5751,CHENG HEOW WONG,2:58:53,INDIVIDUAL,45,None,None,Marathon,None,None,None,1979,SGP,-,Marathon,Male,Seoul Marathon (Korea),17-Mar,2024,International,Marathon
5752,YVONNE ELIZABETH CHEE,3:12:19,INDIVIDUAL,45,None,None,Marathon,None,None,None,1979,SGP,-,Marathon,Female,NN Marathon Rotterdam (Netherlands),14-Apr,2024,International,Marathon
5753,CHEE YONG ONG,2:56:16,INDIVIDUAL,44,None,None,Marathon,None,None,None,1980,SGP,-,Marathon,Male,Seoul Marathon (Korea),17-Mar,2024,International,Marathon


In [1555]:
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:7: SettingWithCopyWarning: 
A value is trying t

In [1556]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


athletes.to_csv('athletes_post_map_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [1557]:
#credentials = service_account.Credentials.from_service_account_file(
#    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
#)

#sql="""
#SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
#FROM `saa-analytics.results.saa_full`
#WHERE STAGE='Final' AND COMPETITION='SEA Game AND RANK='3'
#"""

#benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




In [1558]:
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT YEAR, EVENT, SUB_EVENT, GENDER, NAME, RESULT, RANK, CATEGORY_EVENT, COMPETITION, STAGE, HEAT
FROM `saa-analytics.benchmarks.saa_benchmarks_prod`
WHERE YEAR='2023' AND COMPETITION='Southeast Asian Games' AND (RANK='3' OR RANK='3.0')
"""

SEAG = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1559]:
SEAG

,YEAR,EVENT,SUB_EVENT,GENDER,NAME,RESULT,RANK,CATEGORY_EVENT,COMPETITION,STAGE,HEAT
0,2023,4 x 100m,None,Male,"{0: '\xa0', 1: ' Jonathan Nyepa, Khairul Hafiz...",39.36,3,Relay,Southeast Asian Games,None,None
1,2023,4 x 400m,None,Male,"{0: '\xa0', 1: ' Muhammad Firdaus Bin Mohamad ...",03:08.8,3,Relay,Southeast Asian Games,None,None
2,2023,4 x 100m,None,Female,"{0: '\xa0', 1: ' Azreen Nabila Alias, Nur Afri...",44.58,3,Relay,Southeast Asian Games,None,None
3,2023,4 x 400m,None,Female,"{0: '\xa0', 1: ' Sukanya Janchaona, Benny Nont...",03:39.3,3,Relay,Southeast Asian Games,None,None
4,2023,400m,None,Male,Frederick Ramirez,46.63,3,Sprint,Southeast Asian Games,None,None
...,...,...,...,...,...,...,...,...,...,...,...
75,2023,100m Hurdles,None,Female,Natchaya Chowpakpang,14.23,3,Hurdles,Southeast Asian Games,Heats,Heat 2
76,2023,Marathon,None,Male,Nguyen Thanh Hoang,2:35:49,3.0,Marathon,Southeast Asian Games,None,None
77,2023,Marathon,None,Female,Christine Organiza Hallasgo,2:50:27,3.0,Marathon,Southeast Asian Games,None,None
78,2023,20km Race Walk,None,Female,Kotchaphon Tangsrivong,1:57:11,3.0,20km Race Walk,Southeast Asian Games,None,None


In [1560]:
SEAG_filtered=SEAG[SEAG['HEAT'].isnull() & SEAG['SUB_EVENT'].isnull()]

In [1561]:
benchmarks=SEAG_filtered

In [1562]:
benchmarks

,YEAR,EVENT,SUB_EVENT,GENDER,NAME,RESULT,RANK,CATEGORY_EVENT,COMPETITION,STAGE,HEAT
0,2023,4 x 100m,None,Male,"{0: '\xa0', 1: ' Jonathan Nyepa, Khairul Hafiz...",39.36,3,Relay,Southeast Asian Games,None,None
1,2023,4 x 400m,None,Male,"{0: '\xa0', 1: ' Muhammad Firdaus Bin Mohamad ...",03:08.8,3,Relay,Southeast Asian Games,None,None
2,2023,4 x 100m,None,Female,"{0: '\xa0', 1: ' Azreen Nabila Alias, Nur Afri...",44.58,3,Relay,Southeast Asian Games,None,None
3,2023,4 x 400m,None,Female,"{0: '\xa0', 1: ' Sukanya Janchaona, Benny Nont...",03:39.3,3,Relay,Southeast Asian Games,None,None
4,2023,400m,None,Male,Frederick Ramirez,46.63,3,Sprint,Southeast Asian Games,None,None
5,2023,"10,000m",None,Male,Than Htike Soe,31:25.5,3,Long,Southeast Asian Games,None,None
6,2023,3000m Steeplechase,None,Male,Pandu Sukarya,08:55.0,3,Steeple,Southeast Asian Games,None,None
7,2023,110m Hurdles,None,Male,John Cabang,13.85,3,Hurdles,Southeast Asian Games,None,None
8,2023,400m Hurdles,None,Male,Jun Jie Calvin Quek,50.75,3,Hurdles,Southeast Asian Games,None,None
9,2023,High Jump,None,Male,Farrell Glenn Felix,2.15,3,Jump,Southeast Asian Games,None,None


In [1563]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/127349024.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [1564]:
benchmarks.drop(['YEAR', 'HEAT', 'NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2795758766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmarks.drop(['YEAR', 'HEAT', 'NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [1565]:
benchmarks=benchmarks.reset_index(drop=True)

In [1566]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
            print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
          #  elif 'w' in metric:
                
               
           #     metric1=metric.replace('w', '')
                
           #     output=float(str(metric1))
                
                else:
    
                    output=float(str(metric))
        
        #elif 'w' in metric:
            
                
         #       metric1=metric.replace('w', '')
                             
         #       output=float(str(metric1))
                
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
      #      elif 'marathon' in string and count==2:
                
      #          print('HERE', metric)
                
      #          metric = metric.replace(":", ".", 2)
                
      #          print('METRIC', metric)
                
       #         h,m,s = metric.split(':')            

       #         output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())

               
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [1567]:
def process_benchmarks(df):
    
    for i in range(len(df)):

        rowIndex = df.index[i]

        input_string=df.iloc[rowIndex,0]
    
        metric=df.iloc[rowIndex,3]
    
        if metric==None:
        
            continue
        
        out = convert_time(i, input_string, metric)
        
        print(rowIndex, input_string, out)

    
        df.loc[rowIndex, 'Metric'] = out
    
    return df

In [1568]:
process_benchmarks(benchmarks)

0 4 x 100m 39.36
1 4 x 400m 188.8
2 4 x 100m 44.58
3 4 x 400m 219.3
4 400m 46.63
5 10,000m 1885.5
6 3000m Steeplechase 535.0
7 110m Hurdles 13.85
8 400m Hurdles 50.75
9 High Jump 2.15
10 Shot Put 17.3
11 400m 53.84
12 1500m 266.3
13 10,000m 2131.0
14 400m Hurdles 59.09
15 Triple Jump 13.46
16 200m 21.02
17 800m 113.9
18 5000m 883.4
19 Discus Throw 50.02
20 Javelin Throw 66.2
21 Decathlon 6891.0
22 800m 129.2
23 5000m 1033.6
24 3000m Steeplechase 660.9
25 High Jump 1.73
26 Discus Throw 45.08
27 Heptathlon 5253.0
28 100m 10.45
29 100m 10.45
30 1500m 239.4
31 Pole Vault 5.2
32 Long Jump 7.62
33 Triple Jump 15.7
34 Hammer Throw 59.76
35 100m 11.75
36 200m 23.6
37 100m Hurdles 13.59
38 Pole Vault 4.0
39 Long Jump 6.02
40 Shot Put 14.44
41 Hammer Throw 49.61
42 Javelin Throw 48.31
43 Marathon 9349.0
44 Marathon 10227.0
45 20km Race Walk 7031.0
46 20km Race Walk 6579.0


,EVENT,SUB_EVENT,GENDER,BENCHMARK,Metric
0,4 x 100m,None,Male,39.36,39.36
1,4 x 400m,None,Male,03:08.8,188.80
2,4 x 100m,None,Female,44.58,44.58
3,4 x 400m,None,Female,03:39.3,219.30
4,400m,None,Male,46.63,46.63
5,"10,000m",None,Male,31:25.5,1885.50
6,3000m Steeplechase,None,Male,08:55.0,535.00
7,110m Hurdles,None,Male,13.85,13.85
8,400m Hurdles,None,Male,50.75,50.75
9,High Jump,None,Male,2.15,2.15


In [1569]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,0]
    
    metric=benchmarks.iloc[rowIndex,3]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

0 4 x 100m 39.36
1 4 x 400m 188.8
2 4 x 100m 44.58
3 4 x 400m 219.3
4 400m 46.63
5 10,000m 1885.5
6 3000m Steeplechase 535.0
7 110m Hurdles 13.85
8 400m Hurdles 50.75
9 High Jump 2.15
10 Shot Put 17.3
11 400m 53.84
12 1500m 266.3
13 10,000m 2131.0
14 400m Hurdles 59.09
15 Triple Jump 13.46
16 200m 21.02
17 800m 113.9
18 5000m 883.4
19 Discus Throw 50.02
20 Javelin Throw 66.2
21 Decathlon 6891.0
22 800m 129.2
23 5000m 1033.6
24 3000m Steeplechase 660.9
25 High Jump 1.73
26 Discus Throw 45.08
27 Heptathlon 5253.0
28 100m 10.45
29 100m 10.45
30 1500m 239.4
31 Pole Vault 5.2
32 Long Jump 7.62
33 Triple Jump 15.7
34 Hammer Throw 59.76
35 100m 11.75
36 200m 23.6
37 100m Hurdles 13.59
38 Pole Vault 4.0
39 Long Jump 6.02
40 Shot Put 14.44
41 Hammer Throw 49.61
42 Javelin Throw 48.31
43 Marathon 9349.0
44 Marathon 10227.0
45 20km Race Walk 7031.0
46 20km Race Walk 6579.0


In [1570]:
benchmarks

,EVENT,SUB_EVENT,GENDER,BENCHMARK,Metric
0,4 x 100m,None,Male,39.36,39.36
1,4 x 400m,None,Male,03:08.8,188.80
2,4 x 100m,None,Female,44.58,44.58
3,4 x 400m,None,Female,03:39.3,219.30
4,400m,None,Male,46.63,46.63
5,"10,000m",None,Male,31:25.5,1885.50
6,3000m Steeplechase,None,Male,08:55.0,535.00
7,110m Hurdles,None,Male,13.85,13.85
8,400m Hurdles,None,Male,50.75,50.75
9,High Jump,None,Male,2.15,2.15


In [1571]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put', na=True)

benchmarks.loc[mask, '2%']=benchmarks['Metric']*0.98
benchmarks.loc[mask, '3.5%']=benchmarks['Metric']*0.965
benchmarks.loc[mask, '5%']=benchmarks['Metric']*0.95

benchmarks.loc[~mask, '2%']=benchmarks['Metric']*1.02
benchmarks.loc[~mask, '3.5%']=benchmarks['Metric']*1.035
benchmarks.loc[~mask, '5%']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [1572]:
benchmarks['MAPPED_EVENT']=benchmarks['EVENT'].str.strip()

In [1573]:
for col in benchmarks.columns:
    benchmarks[col] = benchmarks[col].astype(str)
    benchmarks[col] = benchmarks[col].str.replace('\xa0', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\r', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\n', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.strip()


In [1574]:
benchmarks.head(50)

,EVENT,SUB_EVENT,GENDER,BENCHMARK,Metric,2%,3.5%,5%,MAPPED_EVENT
0,4 x 100m,None,Male,39.36,39.36,40.1472,40.73759999999999,41.328,4 x 100m
1,4 x 400m,None,Male,03:08.8,188.8,192.57600000000002,195.408,198.24,4 x 400m
2,4 x 100m,None,Female,44.58,44.58,45.4716,46.140299999999996,46.809,4 x 100m
3,4 x 400m,None,Female,03:39.3,219.3,223.686,226.97549999999998,230.26500000000001,4 x 400m
4,400m,None,Male,46.63,46.63,47.5626,48.26205,48.96150000000001,400m
5,"10,000m",None,Male,31:25.5,1885.5,1923.21,1951.4924999999998,1979.775,"10,000m"
6,3000m Steeplechase,None,Male,08:55.0,535.0,545.7,553.7249999999999,561.75,3000m Steeplechase
7,110m Hurdles,None,Male,13.85,13.85,14.127,14.334749999999998,14.5425,110m Hurdles
8,400m Hurdles,None,Male,50.75,50.75,51.765,52.52625,53.2875,400m Hurdles
9,High Jump,None,Male,2.15,2.15,2.1069999999999998,2.07475,2.0425,High Jump


In [1575]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,MAPPED_EVENT
5079,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,Hammer Throw
5080,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,Pole Vault
5081,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local,
5082,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,
5083,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23941,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local,4 x 400m
23942,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,nan,nan,,Relay,Female,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local,4 x 100m
23943,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,nan,nan,,Relay,Male,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local,4 x 100m
23944,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,nan,nan,,Relay,Female,11th Club ZOOM Kindred Spirit 2024 & PEERS,11/23 to 11/24,2024,Local,4 x 100m


In [1576]:
athletes[athletes['NAME']=='Heng, Richard']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,MAPPED_EVENT
11606,"Heng, Richard",17:02.8,National University Singapore,0,3,Open,Run,5000,,nan,nan,nan,,Long,Male,IVP Track & Field Championships 2024,1/10 to 1/14,2024,Local,5000m
22832,"Heng, Richard",34:11.7,Club Zoom,22,2,Open,Run,10000,,nan,9/5/02,nan,,Long,Male,84th Singapore Open Track & Field,4/18 to 4/19,2024,Local,"10,000m"
22977,"Heng, Richard",35:36.1,National University Singapore,22,4,Open,Run,10000,,nan,5/9/02,nan,,Long,Mixed,SA Allcomers Meet 2,2/25 to 3/3,2024,Local,"10,000m"


In [1577]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [1578]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmarks,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['MAPPED_EVENT', 'GENDER'],
)

In [1579]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,YEAR,REGION,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,2024,Local,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,2024,Local,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,...,2024,Local,4 x 400m,4 x 400m,None,03:08.8,188.8,192.57600000000002,195.408,198.24
20201,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,...,2024,Local,4 x 100m,4 x 100m,None,44.58,44.58,45.4716,46.140299999999996,46.809
20202,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,...,2024,Local,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
20203,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,...,2024,Local,4 x 100m,4 x 100m,None,44.58,44.58,45.4716,46.140299999999996,46.809


In [1580]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,YEAR,REGION,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
1692,Caleb Hia,2:30:15,None,None,193,None,Marathon,None,None,None,...,2024,International,Marathon,Marathon,None,2:35:49,9349.0,9535.98,9676.215,9816.45


In [1581]:
df[df['MAPPED_EVENT']=='10,000m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,YEAR,REGION,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
540,"Goh, Shaun",32:34.2,Club ZOOM,27,1,Open,Run,10000,,nan,...,2024,Local,"10,000m",NaN,NaN,NaN,NaN,NaN,NaN,NaN
604,DARREN CHONG SHUN HAO,37:54:00,SINGAPORE,31,25,None,10000m,10000m,None,None,...,2024,International,"10,000m","10,000m",None,31:25.5,1885.5,1923.21,1951.4924999999998,1979.775
605,LEE VANESSA YING ZHUANG,35:55:00,SINGAPORE,26,8,None,10000m,None,None,None,...,2024,International,"10,000m","10,000m",None,35:31.0,2131.0,2173.62,2205.585,2237.55
606,VANESSA LEE YING ZHUANG,40:21:00,SINGAPORE,26,10,None,10000m,None,None,None,...,2024,International,"10,000m","10,000m",None,35:31.0,2131.0,2173.62,2205.585,2237.55
611,"GOH, SHAUN",33:48:00,Singapore,27,87,None,10000m,None,None,None,...,2024,International,"10,000m","10,000m",None,31:25.5,1885.5,1923.21,1951.4924999999998,1979.775
728,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,...,2024,International,"10,000m","10,000m",None,31:25.5,1885.5,1923.21,1951.4924999999998,1979.775
732,DANIEL LEOW SOON YE,35:10:00,INDIVIDUAL,27,7,None,10000m,None,None,None,...,2024,International,"10,000m","10,000m",None,31:25.5,1885.5,1923.21,1951.4924999999998,1979.775
786,"Rajkumar, Vicknesh",38:00.4,Lacticbuds,32,4,Open,Run,10000,,nan,...,2024,Local,"10,000m",NaN,NaN,NaN,NaN,NaN,NaN,NaN
804,"Lee, Dillon",38:57.3,TeamFabian,29,6,Open,Run,10000,,nan,...,2024,Local,"10,000m",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,"Gurung, Subas",32:09.9,Totality 360,27,1,Open,Run,10000,,nan,...,2024,Local,"10,000m",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1582]:
# replace '-' with NaN

df['RESULT'] = df['RESULT'].replace(regex=r'–', value=np.NaN)
#df['SEED'] = df['SEED'].replace(regex=r'–', value=np.NaN)


In [1583]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,YEAR,REGION,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,2024,Local,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,2024,Local,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,...,2024,Local,4 x 400m,4 x 400m,None,03:08.8,188.8,192.57600000000002,195.408,198.24
20201,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,...,2024,Local,4 x 100m,4 x 100m,None,44.58,44.58,45.4716,46.140299999999996,46.809
20202,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,...,2024,Local,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
20203,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,...,2024,Local,4 x 100m,4 x 100m,None,44.58,44.58,45.4716,46.140299999999996,46.809


In [1584]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,YEAR,REGION,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
1692,Caleb Hia,2:30:15,None,None,193,None,Marathon,None,None,None,...,2024,International,Marathon,Marathon,None,2:35:49,9349.0,9535.98,9676.215,9816.45


In [1585]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


df.to_csv('df_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [1586]:
# Convert results and seed into seconds format

for i in range(len(df)):
    
    result_out=''
    
        
    rowIndex = df.index[i]

    input_string=df.iloc[rowIndex,6]    # event description
    
    metric=df.iloc[rowIndex,1] # result
 #   metric_seed=df.iloc[rowIndex, 1]  #seed
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric=='DNS' or metric=='NH' or metric=='NM' or metric=='FOUL' or metric=='DNF' or metric=='SR' :
        continue
    
    result_out = convert_time(i, input_string, metric)
    print('line', i, input_string, metric, result_out)

    
#    seed_out = convert_time(i, input_string, metric_seed)
         
    df.loc[rowIndex, 'RESULT_CONV'] = result_out
#    df.loc[rowIndex, 'SEED_CONV'] = seed_out


line 0 Hammer Throw 32.40m 32.4
line 1 Pole Vault 4.41m 4.41
line 2 Run 07:38.0 458.0
line 3 Run 15.03 15.03
line 4 Javelin Throw 22.43m 22.43
line 5 Shot Put 8.56m 8.56
line 6 Discus Throw 22.96m 22.96
line 7 Shot Put 8.14m 8.14
line 8 Run 07:00.5 420.5
line 9 Race Walk 28:09.6 1689.6
line 10 Dash 48.68 48.68
line 11 Triple Jump 6.41m 6.41
line 12 Dash 17.32 17.32
line 13 Dash 17.32 17.32
line 14 Long Jump 2.17m 2.17
line 15 Run 02:10.7 130.7
line 16 Dash 11.98 11.98
line 17 Dash 11.98 11.98
line 18 Run 02:27.7 147.7
line 19 Dash 59.82 59.82
line 20 Dash 14.11 14.11
line 21 Dash 14.11 14.11
line 22 Dash 25.38 25.38
line 23 Dash 12.02 12.02
line 24 Dash 12.02 12.02
line 25 Run 02:12.8 132.8
line 26 Run 04:23.9 263.9
line 27 Run 02:20.0 140.0
line 28 Dash 17.18 17.18
line 29 Dash 01:23.7 83.7
line 30 Dash 13.82 13.82
line 31 Dash 13.82 13.82
line 32 Dash 15.06 15.06
line 33 Dash 01:03.8 63.8
line 34 Dash 31.98 31.98
line 35 Dash 12.59 12.59
line 36 Dash 12.59 12.59
line 37 Dash 01:00.0 

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2756656702.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[rowIndex, 'RESULT_CONV'] = result_out


line 2376 100m 00:14.99 14.99
line 2377 100m 00:15.42 15.42
line 2378 4x100m Relay 00:55.32 55.32
line 2379 4x100m Relay 00:57.07 57.07
line 2380 1500m 06:20.85 380.85
line 2381 1500m 06:52.14 412.14
line 2382 4x100m Relay 00:56.33 56.33
line 2383 4x100m Relay 00:55.20 55.2
line 2384 80m Hurdles 00:15.80 15.8
line 2385 800m 03:23.26 203.26
line 2386 800m 03:11.72 191.72
line 2387 800m 02:44.46 164.46
line 2388 800m 02:55.67 175.67
line 2389 800m 02:55.87 175.87
line 2390 800m 03:01.87 181.87
line 2391 Long Jump 3.92 3.92
line 2392 200m 00:31.81 31.81
line 2393 200m 00:29.61 29.61
line 2394 200m 00:29.96 29.96
line 2395 200m 00:30.74 30.74
line 2396 200m 00:30.52 30.52
line 2397 200m 00:29.32 29.32
line 2398 Long Jump 4.09 4.09
line 2399 Long Jump 4.39 4.39
line 2400 Long Jump 3.97 3.97
line 2401 Shot Put 8.08 8.08
line 2402 400m 01:09.3 69.3
line 2403 400m 01:07.0 67.0
line 2404 400m 01:10.7 70.7
line 2405 100m 00:14.41 14.41
line 2406 100m 00:14.17 14.17
line 2407 100m 00:14.42 14.42


line 4897 100m 00:13.68 13.68
line 4898 100m 00:14.19 14.19
line 4899 3000m 13:07.88 787.88
line 4900 3000m 13:55.69 835.69
line 4901 3000m 12:39.98 759.98
line 4902 Javelin 28.04 28.04
line 4903 Javelin 25.36 25.36
line 4904 Javelin 25.69 25.69
line 4905 Triple Jump 9.29 9.29
line 4906 Triple Jump 10.10 10.1
line 4907 800m 02:41.58 161.58
line 4908 800m 02:35.29 155.29
line 4909 800m 02:52.12 172.12
line 4910 200m 00:29.92 29.92
line 4911 200m 00:29.18 29.18
line 4912 200m 00:29.92 29.92
line 4913 200m 00:27.69 27.69
line 4914 200m 00:28.90 28.9
line 4915 Triple Jump 9.47 9.47
line 4916 Triple Jump 10.44 10.44
line 4917 Triple Jump 9.56 9.56
line 4918 100m Hurdles 00:16.79 16.79
line 4919 100m Hurdles 00:17.16 17.16
line 4920 80m Hurdles 00:14.03 14.03
line 4921 80m Hurdles 00:13.36 13.36
line 4922 80m Hurdles 00:14.61 14.61
line 4923 3000m 12:52.40 772.4
line 4924 3000m 12:33.01 753.01
line 4925 3000m 12:37.10 757.1
line 4926 3000m 13:18.21 798.21
line 4927 1500m Race Walk 10:36.72 6

line 7603 Triathlon 16.17 16.17
line 7604 Triathlon 1.79m 16.17
line 7605 Triathlon 4.55m 16.17
line 7606 Dash 15.41 15.41
line 7607 Triathlon 2.46m 15.41
line 7608 Triathlon 16.36 16.36
line 7610 Triathlon 2.49m 16.36
line 7611 Triathlon 14.14 14.14
line 7612 Dash 13.73 13.73
line 7613 Triathlon 11.57m 13.73
line 7614 Dash 13.03 13.03
line 7615 Dash 38.96 38.96
line 7616 Dash 14.75 14.75
line 7617 Dash 14.53 14.53
line 7618 Dash 14.48 14.48
line 7619 Dash 13.74 13.74
line 7620 Dash 14.18 14.18
line 7621 Dash 13.25 13.25
line 7622 Dash 12.46 12.46
line 7623 Dash 15.94 15.94
line 7624 Triathlon 6.97m 15.94
line 7625 Triathlon 15.62 15.62
line 7626 Triathlon 2.04m 15.62
line 7627 Dash 14.77 14.77
line 7628 Dash 12.44 12.44
line 7629 Dash 14.31 14.31
line 7630 Dash 27.35 27.35
line 7631 Dash 14 14.0
line 7632 Dash 01:21.0 81.0
line 7633 Dash 15.02 15.02
line 7634 Dash 01:21.6 81.6
line 7635 Dash 13.27 13.27
line 7636 Dash 01:20.6 80.6
line 7637 Triathlon 1.74m 80.6
line 7638 Triathlon 4.8

line 10219 Dash 31.83 31.83
line 10220 Dash 13.85 13.85
line 10221 Dash 13.85 13.85
line 10222 Dash 14.79 14.79
line 10223 Dash 30.02 30.02
line 10225 Dash 14.24 14.24
line 10226 Dash 29.25 29.25
line 10227 High Jump 1.25m 1.25
line 10228 Dash 15.16 15.16
line 10229 Hurdles 01:22.2 82.2
line 10230 Dash 15.61 15.61
line 10231 Dash 32.88 32.88
line 10232 Run 02:53.2 173.2
line 10233 Dash 15.38 15.38
line 10234 Long Jump 3.78m 3.78
line 10235 Long Jump 4.10m 4.1
line 10236 Run 07:16.1 436.1
line 10237 Run 06:30.1 390.1
line 10238 Run 06:54.9 414.9
line 10239 Run 08:23.2 503.2
line 10240 Shot Put 5.79m 5.79
line 10241 Javelin Throw 13.36m 13.36
line 10242 Javelin Throw 10.65m 10.65
line 10243 Javelin Throw 23.54m 23.54
line 10244 Javelin Throw 8.79m 8.79
line 10245 Dash 12.42 12.42
line 10246 Dash 12.23 12.23
line 10247 Long Jump 4.37m 4.37
line 10248 Long Jump 4.07m 4.07
line 10249 Long Jump 3.90m 3.9
line 10250 Long Jump 4.35m 4.35
line 10251 Dash 31.35 31.35
line 10252 Dash 37.5 37.5
li

line 12879 Dash 12.41 12.41
line 12880 Mile Run 05:47.5 347.5
line 12881 Run 05:39.4 339.4
line 12882 Dash 27.29 27.29
line 12883 Dash 15.06 15.06
line 12884 Dash 33.73 33.73
line 12885 Dash 26.68 26.68
line 12886 Dash 01:02.9 62.9
line 12887 Dash 13.68 13.68
line 12888 Dash 13.68 13.68
line 12889 Dash 01:04.6 64.6
line 12890 Shot Put 4.77m 4.77
line 12891 Dash 01:09.5 69.5
line 12892 Dash 31.08 31.08
line 12893 Dash 01:15.2 75.2
line 12894 Run 06:39.8 399.8
line 12895 Run 02:53.8 173.8
line 12896 Hurdles 33.66 33.66
line 12897 Dash 14.39 14.39
line 12898 Dash 14.39 14.39
line 12899 Dash 31.47 31.47
line 12900 Dash 01:16.7 76.7
line 12901 Dash 01:16.1 76.1
line 12902 Dash 14.43 14.43
line 12903 Dash 14.43 14.43
line 12904 Dash 30.47 30.47
line 12905 Dash 15.08 15.08
line 12906 Dash 15.08 15.08
line 12907 Dash 13 13.0
line 12908 Dash 13 13.0
line 12909 Dash 12.93 12.93
line 12910 Dash 12.93 12.93
line 12911 Dash 26.01 26.01
line 12912 Dash 30.87 30.87
line 12913 Dash 14.77 14.77
line 12

line 15572 Discus Throw 28.00m 28.0
line 15573 Shot Put 8.12m 8.12
line 15574 Hurdles 01:27.4 87.4
line 15575 Dash 01:16.0 76.0
line 15576 Run 05:44.8 344.8
line 15577 Dash 28.63 28.63
line 15578 Dash 13.77 13.77
line 15579 Dash 13.69 13.69
line 15580 Long Jump 3.88m 3.88
line 15581 Long Jump 3.98m 3.98
line 15582 Shot Put 9.39m 9.39
line 15583 Shot Put 9.00m 9.0
line 15584 Hurdles 01:19.3 79.3
line 15585 Hurdles 01:19.7 79.7
line 15586 Long Jump 4.82m 4.82
line 15587 Dash 14.34 14.34
line 15588 Hurdles 17.24 17.24
line 15589 Hurdles 17.31 17.31
line 15590 Long Jump 4.57m 4.57
line 15591 Long Jump 3.72m 3.72
line 15592 Long Jump 3.69m 3.69
line 15593 Triple Jump 9.63m 9.63
line 15594 High Jump 1.35m 1.35
line 15595 Triple Jump 9.68m 9.68
line 15596 Long Jump 4.21m 4.21
line 15597 High Jump 1.30m 1.3
line 15598 Triple Jump 9.07m 9.07
line 15599 Triple Jump 9.10m 9.1
line 15600 High Jump 1.30m 1.3
line 15601 Long Jump 4.29m 4.29
line 15602 High Jump 1.35m 1.35
line 15603 Hurdles 01:32.4 

line 18211 Dash 24.59 24.59
line 18212 Dash 24.33 24.33
line 18213 Dash 24.63 24.63
line 18214 Dash 11.73 11.73
line 18215 Dash 11.73 11.73
line 18216 Dash 11.82 11.82
line 18217 Dash 11.82 11.82
line 18218 Triple Jump 12.22m 12.22
line 18220 Dash 11.52 11.52
line 18221 Dash 11.52 11.52
line 18222 Dash 23.96 23.96
line 18223 Javelin Throw 36.35m 36.35
line 18224 Dash 26.76 26.76
line 18225 Dash 26.64 26.64
line 18226 Dash 12.49 12.49
line 18227 Dash 12.7 12.7
line 18228 Dash 34.19 34.19
line 18229 Run 01:56.9 116.9
line 18230 Run 04:20.8 260.8
line 18231 Dash 10.93 10.93
line 18232 Dash 10.93 10.93
line 18233 Dash 22.96 22.96
line 18234 Dash 10.81 10.81
line 18235 Dash 10.81 10.81
line 18236 Dash 01:07.6 67.6
line 18237 Dash 01:12.0 72.0
line 18238 Dash 14.41 14.41
line 18239 Dash 31.43 31.43
line 18240 Dash 14.6 14.6
line 18241 Dash 31.13 31.13
line 18242 Dash 53.06 53.06
line 18243 Hurdles 25.2 25.2
line 18244 Dash 24.67 24.67
line 18245 Dash 01:11.1 71.1
line 18246 Dash 01:12.1 72.1

In [1587]:
#df["AGE"].fillna(0, inplace=True)
#df['AGE'] = df['AGE'].astype('float')

In [1588]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,REGION,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,Local,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772,32.4
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,Local,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995,4.41
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,458.0
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.03
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,...,Local,4 x 400m,4 x 400m,None,03:08.8,188.8,192.57600000000002,195.408,198.24,252.1
20201,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,...,Local,4 x 100m,4 x 100m,None,44.58,44.58,45.4716,46.140299999999996,46.809,59.79
20202,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,...,Local,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,46.33
20203,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,...,Local,4 x 100m,4 x 100m,None,44.58,44.58,45.4716,46.140299999999996,46.809,62.1


In [1589]:
# Choose SEED if better than RESULT

#condition1=df['SEED_CONV']>df['RESULT_CONV']
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


#df['RESULT_BEST']=df['SEED_CONV'].where((condition1 & condition2)|(condition3 & condition4), df['RESULT_CONV'].values)

df['RESULT_BEST'] = df['RESULT_CONV']

In [1590]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772,32.4,32.4
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995,4.41,4.41
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,458.0,458.0
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.03,15.03
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.43,22.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,...,4 x 400m,4 x 400m,None,03:08.8,188.8,192.57600000000002,195.408,198.24,252.1,252.1
20201,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,...,4 x 100m,4 x 100m,None,44.58,44.58,45.4716,46.140299999999996,46.809,59.79,59.79
20202,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,...,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,46.33,46.33
20203,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,...,4 x 100m,4 x 100m,None,44.58,44.58,45.4716,46.140299999999996,46.809,62.1,62.1


In [1591]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
1692,Caleb Hia,2:30:15,None,None,193,None,Marathon,None,None,None,...,Marathon,Marathon,None,2:35:49,9349.0,9535.98,9676.215,9816.45,9015.0,9015.0


In [1592]:
# Change to numeric

df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']] = df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']].apply(pd.to_numeric)

In [1593]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_BEST']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_BEST']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_BEST']-df['5%']
df.loc[mask, 'Delta_Benchmark'] = df['RESULT_BEST']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
df=df.loc[df['COMPETITION']!='Southeast Asian Games']

In [1594]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['Metric']*100

In [1595]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


df.to_csv('df_test_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [1596]:
df[df['NAME']=='LEE, VANESSA']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
581,"LEE, VANESSA",11:09.2,SINGAPORE,26,1,Open,Steeplechase,3000,,nan,...,674.118,684.0315,693.945,669.2,669.2,4.918,14.8315,24.745,-8.3,3.744137
582,"LEE, VANESSA",04:51.7,SINGAPORE,26,3,Open,Run,1500,,nan,...,271.626,275.6205,279.615,291.7,291.7,-20.074,-16.0795,-12.085,-25.4,-4.538115


In [1597]:
'''

# Correct some of the known name variations
# Need to fix 2024 NSG names extracts (extracting Q/q with name)

df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()



df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie El Redha Ang El Redha Ang', value='Norhisham, Jamie El Redha Ang')



df['NAME'] = df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'TAN, SHEE RU, ELIZABETH-A', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Elizabeth-Ann Tan', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shee Ru, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')

df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc Brian Brian', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Marc Brian Louis', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Brian louis, Marc', value='Louis, Marc Brian')




df['NAME'] = df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan, Jun Jie')
df['NAME'] = df['NAME'].replace(regex=r'Tan Jun Jie', value='Tan, Jun Jie')

df['NAME'] = df['NAME'].replace(regex=r'Shanti Veronica Pereira', value='PEREIRA, VERONICA SHANTI')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, YEE LING, BERNICE', value='Liew, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, BERNICE YEE LING', value='Liew, Bernice')

df['NAME'] = df['NAME'].replace(regex=r'Mark Lee', value='LEE, MARK REN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Mark', value='LEE, MARK REN')






df['NAME'] = df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')




df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')
df['NAME'] = df['NAME'].replace(regex=r'YAP, SEAN YEE', value='Yap, Sean Yee')
df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie', value='Norhisham, Jamie El Redha Ang')

df['NAME'] = df['NAME'].replace(regex=r'ALEXIS TEO PEI QI', value='Teo, Alexis')



df['NAME'] = df['NAME'].replace(regex=r'Enriquez, Roxanne Rose Zulueta', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROXANNE ROSE ZULUETA', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROSE ZULUETA, ROXANN', value='ROSE ZULUETA ENRIQUE, ROXANNE')

df['NAME'] = df['NAME'].replace(regex=r'Goh, Clara', value='GOH, CLARA')

df['NAME'] = df['NAME'].replace(regex=r'WONG JIN ROU', value='Wong, Jin Rou')



df['NAME'] = df['NAME'].replace(regex=r'THANA RAJAN, S/O, THIRUBEN', value='THANA RAJAN, THIRUBEN  S/O')
df['NAME'] = df['NAME'].replace(regex=r'Rajan Thiruben Thana', value='THANA RAJAN, THIRUBEN  S/O')


df['NAME'] = df['NAME'].replace(regex=r'Raam Kumar Muthukumaran', value='Muthukumaran, Raam Kumar')

df['NAME'] = df['NAME'].replace(regex=r'Chia Wei En, Shawn', value='Chia, Shawn')



df['NAME'] = df['NAME'].replace(regex=r'QUEK XIN YONG JEREMIAH', value='QUEK, JEREMIAH')
df['NAME'] = df['NAME'].replace(regex=r'quek xin yong jeremi', value='QUEK, JEREMIAH')


df['NAME'] = df['NAME'].replace(regex=r'Chiu, Jingwen', value='Chiu, Jing Wen')



df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')


df['NAME'] = df['NAME'].replace(regex=r'LOW CHUAN YI', value='Low, Chuan Yi')



df['NAME'] = df['NAME'].replace(regex=r'Rachel Yang Bing Jie', value='YANG, RACHEL')

df['NAME'] = df['NAME'].replace(regex=r'Jun Yu Low', value='Low, Jun Yu')
df['NAME'] = df['NAME'].replace(regex=r'Low Jun Yu', value='Low, Jun Yu')

df['NAME'] = df['NAME'].replace(regex=r'Dewey Ng', value='Ng, Dewey')
df['NAME'] = df['NAME'].replace(regex=r'NG, DEWEY', value='Ng, Dewey')

df['NAME'] = df['NAME'].replace(regex=r'., Brendon Ting Li King', value='Ting Li King, Brandon')
df['NAME'] = df['NAME'].replace(regex=r'TAN KIAN SWEE ISAAC', value='Tan, Isaac')
df['NAME'] = df['NAME'].replace(regex=r'TAN, KIAN SWEE ISAAC', value='Tan, Isaac')

df['NAME'] = df['NAME'].replace(regex=r'GAO SHENGWEI', value='Gao, Shengwei')
df['NAME'] = df['NAME'].replace(regex=r'NEO SHYH KAI', value='Neo, Shyh Kai')
df['NAME'] = df['NAME'].replace(regex=r'ANGEL LIM YU XIN', value='Lim, Yuxin Angel')
df['NAME'] = df['NAME'].replace(regex=r'LAUREL, LIM JIA EN', value='LAUREL LIM JIA EN')

df['NAME'] = df['NAME'].replace(regex=r'LOH DING RONG, ANSON', value='Loh, Anson')

df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')
df['NAME'] = df['NAME'].replace(regex=r'NG JUN HENG, RYAN', value='Ng, Ryan')
df['NAME'] = df['NAME'].replace(regex=r'LIM WEE ENG', value='Lim, Wee Eng')


df['NAME'] = df['NAME'].replace(regex=r'Chen Xiang Ang', value='Ang, Chen Xiang')

df['NAME'] = df['NAME'].replace(regex=r'CHUA JE-AN, GARRETT', value='Chua, Garrett')
df['NAME'] = df['NAME'].replace(regex=r'Garrett Chua', value='Chua, Garrett')


df['NAME'] = df['NAME'].replace(regex=r'Looi, Yan Ning Samantha', value='LOOI YAN NING, SAMANTHA')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Sheila Qi Le', value='Lim Qi Le, Sheila')
df['NAME'] = df['NAME'].replace(regex=r'NG, CHIN HUI', value='Ng, Chin Hui')
df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')


df['NAME'] = df['NAME'].replace(regex=r'Edsel Poh', value='Poh, Edsel')
df['NAME'] = df['NAME'].replace(regex=r'Gautam, Kulandaiveludayar Se', value='Kulandaiveludayar Se, Gautam')



df['NAME'] = df['NAME'].replace(regex=r'Hanim, Asmah', value='Asmah, Hanim')
df['NAME'] = df['NAME'].replace(regex=r'YOON SAE YOUNG', value='Yoon, Sae Young')
df['NAME'] = df['NAME'].replace(regex=r'Wong, Alexis Li Xuan', value='Wong, Alexis')

df['NAME'] = df['NAME'].replace(regex=r'TAN JUN ERN KEAN', value='Tan Jun Ern Kean')

df['NAME'] = df['NAME'].replace(regex=r'JOEY LIM ZI YI', value='Lim, Joey')
df['NAME'] = df['NAME'].replace(regex=r'Bernie Chong Yi Shan', value='Chong, Bernie')

df['NAME'] = df['NAME'].replace(regex=r'Tan Ting Jun', value='Tan, Ting Jun')
df['NAME'] = df['NAME'].replace(regex=r'Xiao, Hongyang', value='Xiao, Hong Yang')
df['NAME'] = df['NAME'].replace(regex=r'Reiner Leong Rong Xi', value='Leong, Reiner')


df['NAME'] = df['NAME'].replace(regex=r'CHEN JING YING JANELLE', value='Chen, Janelle')
df['NAME'] = df['NAME'].replace(regex=r'Toh Wei De', value='Toh, Wei De')
#df['NAME'] = df['NAME'].replace(regex=r'SheikAluu\'udin\,\sSkeik\sFayz', value='Sheik\sAluu\'udin\,\sSkeik\sFayz')

df['NAME'] = df['NAME'].replace(regex=r'kapil, Arnav', value='Kapil, Arnav')
df['NAME'] = df['NAME'].replace(regex=r'KIRAN, SEBASTIAN JEHUNATHAN', value='Sebastian Jehunathan, Kiran')
df['NAME'] = df['NAME'].replace(regex=r'LAU, REINA', value='Lau, Reina')
df['NAME'] = df['NAME'].replace(regex=r'Lau Reina', value='Lau, Reina')


df['NAME'] = df['NAME'].replace(regex=r'Si Hui, Amanda, Loo', value='Amanda, Loo Si Hui')
df['NAME'] = df['NAME'].replace(regex=r'Zou Aolin q', value='Zou Aolin')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')

df['NAME'] = df['NAME'].replace(regex=r'NEO EE , NIN', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Neo Ee Nin', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Emma Tan Yi Min', value='TAN, EMMA YI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Emma Yi Min', value='TAN, EMMA YI MIN')

df['NAME'] = df['NAME'].replace(regex=r'WONG, JIN ROU', value='Wong, Jin Rou')
df['NAME'] = df['NAME'].replace(regex=r'Seah, Michelle Xin Yi', value='Seah, Michelle')
df['NAME'] = df['NAME'].replace(regex=r'Jing Wen, Chiu', value='Chiu, Jing Wen')
df['NAME'] = df['NAME'].replace(regex=r'NG, ROY', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'Ng, Kai Leong Roy', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'., Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'., Cheong Yin Yern Jose', value='Joseph Cheong Yin Yern')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')
df['NAME'] = df['NAME'].replace(regex=r'Hongyang, Xiao', value='XIAO, HONGYANG')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shou Yi Rei (Chen Shouyi)', value='Tan Shou Yi Rei')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee Yuxi Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Low, Jonathan', value='Low Jun Jie, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'Murali, Sham', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'SHANJIE, MURALI SHAM KUMAR', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'Koh, Aaron Xu Qing', value='Koh, Aaron')

df['NAME'] = df['NAME'].replace(regex=r'Tan Shou Yi Rei (Chen Shouyi)', value='Tan Shou Yi Rei')


df['NAME'] = df['NAME'].replace(regex=r'Mook, He Jun', value='HE, MOOK JUN')
df['NAME'] = df['NAME'].replace(regex=r'Leong, Lorraine', value='Leong Jia En, Lorraine')
df['NAME'] = df['NAME'].replace(regex=r'Joseph Cheong Yin Yern', value='Cheong, Joseph')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Junn Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'Sheik, Fayz', value='Sheik Alau\'ddin, Sheik Fayz')
df['NAME'] = df['NAME'].replace(regex=r'Keondre Liew Zhen Kai', value='Liew, Keondre Zhen Kai')




df['NAME'] = df['NAME'].replace(regex=r'TOH, TEDD', value='Toh Jun Xi, Tedd')



df['NAME'] = df['NAME'].replace(regex=r'Chuhan Zhong', value='ZHONG, CHUHAN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Daniel', value='Lee Zheng Xian, Daniel Christopher')
df['NAME'] = df['NAME'].replace(regex=r'GOH, YEN YOUNG, AMELIA', value='Goh Yen Young Amelia')
df['NAME'] = df['NAME'].replace(regex=r'Liew, Keondre Zhen Kai', value='LIEW, KEONDRE')
df['NAME'] = df['NAME'].replace(regex=r'Janelle, Lum', value='Lum Wai Yan, Janelle')


df['NAME'] = df['NAME'].replace(regex=r'Teh Ying Shan', value='Teh, Ying Shan')

df['NAME'] = df['NAME'].replace(regex=r'Mayah Elizabeth Kaufmann Q', value='Mayah Elizabeth Kaufmann')


df['NAME'] = df['NAME'].replace(regex=r'TNG, KAI XIN', value='Tng Kai Xin')
# Correct some of the known name variations
# Need to fix 2024 NSG names extracts (extracting Q/q with name)

df['NAME'] = df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
df['NAME'] = df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
df['NAME'] = df['NAME'].replace(regex=r'Ryan Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')

df['NAME'] = df['NAME'].replace(regex=r'Ryan Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')


df['NAME'] = df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'TAN, SHEE RU, ELIZABETH-A', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Elizabeth-Ann Tan', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shee Ru, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')

df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc Brian Brian', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Marc Brian Louis', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Brian louis, Marc', value='Louis, Marc Brian')




df['NAME'] = df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan, Jun Jie')
df['NAME'] = df['NAME'].replace(regex=r'Tan Jun Jie', value='Tan, Jun Jie')

df['NAME'] = df['NAME'].replace(regex=r'Shanti Veronica Pereira', value='PEREIRA, VERONICA SHANTI')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, YEE LING, BERNICE', value='Liew, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, BERNICE YEE LING', value='Liew, Bernice')

df['NAME'] = df['NAME'].replace(regex=r'Mark Lee', value='LEE, MARK REN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Mark', value='LEE, MARK REN')





df['NAME'] = df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')



df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')
df['NAME'] = df['NAME'].replace(regex=r'YAP, SEAN YEE', value='Yap, Sean Yee')

df['NAME'] = df['NAME'].replace(regex=r'ALEXIS TEO PEI QI', value='Teo Pei Qi, Alexis')
df['NAME'] = df['NAME'].replace(regex=r'Alexis Teo Pei Qi', value='Teo Pei Qi, Alexis')
df['NAME'] = df['NAME'].replace(regex=r'Teo Pei Qi, Alexis Q', value='Teo Pei Qi, Alexis')




df['NAME'] = df['NAME'].replace(regex=r'Enriquez, Roxanne Rose Zulueta', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROXANNE ROSE ZULUETA', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROSE ZULUETA, ROXANN', value='ROSE ZULUETA ENRIQUE, ROXANNE')

df['NAME'] = df['NAME'].replace(regex=r'Goh, Clara', value='GOH, CLARA')

df['NAME'] = df['NAME'].replace(regex=r'WONG JIN ROU', value='Wong, Jin Rou')



df['NAME'] = df['NAME'].replace(regex=r'THANA RAJAN, S/O, THIRUBEN', value='THANA RAJAN, THIRUBEN  S/O')
df['NAME'] = df['NAME'].replace(regex=r'Rajan Thiruben Thana', value='THANA RAJAN, THIRUBEN  S/O')
df['NAME'] = df['NAME'].replace(regex=r'Thana Rajan, Thiruben', value='THANA RAJAN, THIRUBEN  S/O')



df['NAME'] = df['NAME'].replace(regex=r'Raam Kumar Muthukumaran', value='Muthukumaran, Raam Kumar')

df['NAME'] = df['NAME'].replace(regex=r'CHIA, WEI EN, SHAWN', value='Chia, Shawn')


df['NAME'] = df['NAME'].replace(regex=r'QUEK XIN YONG JEREMIAH', value='QUEK, JEREMIAH')
df['NAME'] = df['NAME'].replace(regex=r'quek xin yong jeremi', value='QUEK, JEREMIAH')


df['NAME'] = df['NAME'].replace(regex=r'Chiu, Jingwen', value='Chiu, Jing Wen')



df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')


df['NAME'] = df['NAME'].replace(regex=r'LOW CHUAN YI', value='Low, Chuan Yi')

df['NAME'] = df['NAME'].replace(regex=r'Rachel Yang Bing Jie', value='YANG, RACHEL')


df['NAME'] = df['NAME'].replace(regex=r'Dewey Ng', value='Ng, Dewey')
df['NAME'] = df['NAME'].replace(regex=r'NG, DEWEY', value='Ng, Dewey')

df['NAME'] = df['NAME'].replace(regex=r'., Brendon Ting Li King', value='Ting Li King, Brandon')
df['NAME'] = df['NAME'].replace(regex=r'TAN KIAN SWEE ISAAC', value='Tan, Isaac')
df['NAME'] = df['NAME'].replace(regex=r'TAN, KIAN SWEE ISAAC', value='Tan, Isaac')

df['NAME'] = df['NAME'].replace(regex=r'GAO SHENGWEI', value='Gao, Shengwei')
df['NAME'] = df['NAME'].replace(regex=r'NEO SHYH KAI', value='Neo, Shyh Kai')
df['NAME'] = df['NAME'].replace(regex=r'ANGEL LIM YU XIN', value='Lim, Yuxin Angel')
df['NAME'] = df['NAME'].replace(regex=r'LAUREL, LIM JIA EN', value='LAUREL LIM JIA EN')

df['NAME'] = df['NAME'].replace(regex=r'LOH DING RONG, ANSON', value='Loh, Anson')

df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')
df['NAME'] = df['NAME'].replace(regex=r'NG JUN HENG, RYAN', value='Ng, Ryan')
df['NAME'] = df['NAME'].replace(regex=r'LIM WEE ENG', value='Lim, Wee Eng')


df['NAME'] = df['NAME'].replace(regex=r'Chen Xiang Ang', value='Ang, Chen Xiang')

df['NAME'] = df['NAME'].replace(regex=r'CHUA JE-AN, GARRETT', value='Chua, Garrett')
df['NAME'] = df['NAME'].replace(regex=r'Garrett Chua', value='Chua, Garrett')


df['NAME'] = df['NAME'].replace(regex=r'Looi, Yan Ning Samantha', value='LOOI YAN NING, SAMANTHA')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Sheila Qi Le', value='Lim Qi Le, Sheila')
df['NAME'] = df['NAME'].replace(regex=r'NG, CHIN HUI', value='Ng, Chin Hui')
df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')

df['NAME'] = df['NAME'].replace(regex=r'CHUA HSIN-WEN CLARA', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua Hsin-Wen Clara', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua Hsin-Wen Clara Q', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'HSIN-WEN CLARA, Chua', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'CHUA HSIN-WEN CLARA', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua, Clara Q', value='Chua Hsin-Wen, Clara')


df['NAME'] = df['NAME'].replace(regex=r'Edsel Poh', value='Poh, Edsel')
df['NAME'] = df['NAME'].replace(regex=r'Gautam, Kulandaiveludayar Se', value='Kulandaiveludayar Se, Gautam')


df['NAME'] = df['NAME'].replace(regex=r'Hanim, Asmah', value='Asmah, Hanim')
df['NAME'] = df['NAME'].replace(regex=r'YOON SAE YOUNG', value='Yoon, Sae Young')
df['NAME'] = df['NAME'].replace(regex=r'Wong, Alexis Li Xuan', value='Wong, Alexis')

df['NAME'] = df['NAME'].replace(regex=r'TAN JUN ERN KEAN', value='Tan Jun Ern Kean')

df['NAME'] = df['NAME'].replace(regex=r'JOEY LIM ZI YI', value='Lim, Joey')
df['NAME'] = df['NAME'].replace(regex=r'Bernie Chong Yi Shan', value='Chong, Bernie')

df['NAME'] = df['NAME'].replace(regex=r'Tan Ting Jun', value='Tan, Ting Jun')
df['NAME'] = df['NAME'].replace(regex=r'Xiao, Hongyang', value='Xiao, Hong Yang')
df['NAME'] = df['NAME'].replace(regex=r'Reiner Leong Rong Xi', value='Leong, Reiner')


df['NAME'] = df['NAME'].replace(regex=r'CHEN JING YING JANELLE', value='Chen, Janelle')
df['NAME'] = df['NAME'].replace(regex=r'Toh Wei De', value='Toh, Wei De')
#df['NAME'] = df['NAME'].replace(regex=r'SheikAluu\'udin\,\sSkeik\sFayz', value='Sheik\sAluu\'udin\,\sSkeik\sFayz')

df['NAME'] = df['NAME'].replace(regex=r'kapil, Arnav', value='Kapil, Arnav')
df['NAME'] = df['NAME'].replace(regex=r'KIRAN, SEBASTIAN JEHUNATHAN', value='Sebastian Jehunathan, Kiran')
df['NAME'] = df['NAME'].replace(regex=r'LAU, REINA', value='Lau, Reina')
df['NAME'] = df['NAME'].replace(regex=r'Lau Reina', value='Lau, Reina')


df['NAME'] = df['NAME'].replace(regex=r'Zou Aolin q', value='Zou Aolin')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')

df['NAME'] = df['NAME'].replace(regex=r'NEO EE , NIN', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Neo Ee Nin', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Emma Tan Yi Min', value='TAN, EMMA YI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Emma Yi Min', value='TAN, EMMA YI MIN')


df['NAME'] = df['NAME'].replace(regex=r'Si Hui, Amanda, Loo', value='Amanda, Loo Si Hui')
df['NAME'] = df['NAME'].replace(regex=r'Si Hui Amanda, Loo', value='Amanda, Loo Si Hui')



df['NAME'] = df['NAME'].replace(regex=r'WONG, JIN ROU', value='Wong, Jin Rou')
df['NAME'] = df['NAME'].replace(regex=r'Seah, Michelle Xin Yi', value='Seah, Michelle')
df['NAME'] = df['NAME'].replace(regex=r'Jing Wen, Chiu', value='Chiu, Jing Wen')
df['NAME'] = df['NAME'].replace(regex=r'NG, ROY', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'Ng, Kai Leong Roy', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'., Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'., Cheong Yin Yern Jose', value='Joseph Cheong Yin Yern')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')
df['NAME'] = df['NAME'].replace(regex=r'Hongyang, Xiao', value='XIAO, HONGYANG')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shou Yi Rei (Chen Shouyi)', value='Tan Shou Yi Rei')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee Yuxi Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Low, Jonathan', value='Low Jun Jie, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'Murali, Sham', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'SHANJIE, MURALI SHAM KUMAR', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'Koh, Aaron Xu Qing', value='Koh, Aaron')

df['NAME'] = df['NAME'].replace(regex=r'^Tan\sShou\sYi\sRei$', value='Tan Shou Yi Rei (Chen Shouyi)')



df['NAME'] = df['NAME'].replace(regex=r'KUMAR, VIRESH', value='Kumar, Viresh')
df['NAME'] = df['NAME'].replace(regex=r'KUMAR, S VIRESH', value='Kumar, Viresh')
df['NAME'] = df['NAME'].replace(regex=r's Viresh Kumar', value='Kumar, Viresh')
df['NAME'] = df['NAME'].replace(regex=r'Kumar, S Viresh', value='Kumar, Viresh')


df['NAME'] = df['NAME'].replace(regex=r'Mook, He Jun', value='HE, MOOK JUN')
df['NAME'] = df['NAME'].replace(regex=r'Leong, Lorraine', value='Leong Jia En, Lorraine')
df['NAME'] = df['NAME'].replace(regex=r'Joseph Cheong Yin Yern', value='Cheong, Joseph')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Junn Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'Sheik, Fayz', value='Sheik Alau\'ddin, Sheik Fayz')
df['NAME'] = df['NAME'].replace(regex=r'Keondre Liew Zhen Kai', value='Liew, Keondre Zhen Kai')




df['NAME'] = df['NAME'].replace(regex=r'TOH, TEDD', value='Toh Jun Xi, Tedd')
df['NAME'] = df['NAME'].replace(regex=r'Toh, Tedd', value='Toh Jun Xi, Tedd')





df['NAME'] = df['NAME'].replace(regex=r'Chuhan Zhong', value='ZHONG, CHUHAN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Daniel', value='Lee Zheng Xian, Daniel Christopher')



df['NAME'] = df['NAME'].replace(regex=r'GOH, YEN YOUNG, AMELIA', value='Goh Yen Young Amelia')
df['NAME'] = df['NAME'].replace(regex=r'lim, eiffer', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'Eiffer, Lim Ying Qi', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'Liew, Keondre Zhen Kai', value='LIEW, KEONDRE')
df['NAME'] = df['NAME'].replace(regex=r'Janelle, Lum', value='Lum Wai Yan, Janelle')
df['NAME'] = df['NAME'].replace(regex=r'Teh Ying Shan', value='Teh, Ying Shan')
df['NAME'] = df['NAME'].replace(regex=r'Ying Shan, Teh', value='Teh, Ying Shan')


df['NAME'] = df['NAME'].replace(regex=r'Mayah Elizabeth Kaufmann Q', value='Mayah Elizabeth Kaufmann')
df['NAME'] = df['NAME'].replace(regex=r'TNG, KAI XIN', value='Tng Kai Xin')
df['NAME'] = df['NAME'].replace(regex=r'YANG, BOBBI', value='Bobbi Victoria Yang Yen (Weng Xuan)')

df['NAME'] = df['NAME'].replace(regex=r'Ng, Si En Tabitha', value='Ng Si En, Tabitha')
df['NAME'] = df['NAME'].replace(regex=r'Hoh Ying Min', value='Hoh, Ying Min')
df['NAME'] = df['NAME'].replace(regex=r'Tedd Toh', value='Toh Jun Xi, Tedd')
df['NAME'] = df['NAME'].replace(regex=r'Irfan Qabeel Mohamman', value='Mohammad Irfan, Qabeel')
df['NAME'] = df['NAME'].replace(regex=r'Sebastian, Kiran', value='Sebastian Jehunathan, Kiran')
df['NAME'] = df['NAME'].replace(regex=r'TANG, CAYMAN', value='Tang Kai Sheng, Cayman')
df['NAME'] = df['NAME'].replace(regex=r'ZHONG, CHUHAN', value='Zhong, Chuhan')

df['NAME'] = df['NAME'].replace(regex=r'Reuben Lee Siong En', value='LEE SIONG EN, REUBEN RAINER')
df['NAME'] = df['NAME'].replace(regex=r'Tam, Joash', value='Tam Jong-Hng, Joash')
df['NAME'] = df['NAME'].replace(regex=r'Pooran Sankaresh Dharmaraj', value='Pooran, Dharmaraj')

df['NAME'] = df['NAME'].replace(regex=r'Meena, Girish Chandramohan', value='Chandramohan Meena, Girish')
df['NAME'] = df['NAME'].replace(regex=r'Wong Jin Rou', value='Wong, Jin Rou')
df['NAME'] = df['NAME'].replace(regex=r'MOHHAMMAD IRFAN, QABEEL', value='Mohammad Irfan, Qabeel')
df['NAME'] = df['NAME'].replace(regex=r'BIN MD, MOHAMMAD IRFAN QABEE', value='Mohammad Irfan, Qabeel')



df['NAME'] = df['NAME'].replace(regex=r'TAM, JOASH', value='Tam Jong-Hng, Joash')
df['NAME'] = df['NAME'].replace(regex=r'Verboon Lara', value='Verboon, Lara')
df['NAME'] = df['NAME'].replace(regex=r'Verboon , Lara', value='Verboon, Lara')

df['NAME'] = df['NAME'].replace(regex=r'Megan Anne Ying Ka Mun Q', value='Megan Anne Ying Ka Mun')
df['NAME'] = df['NAME'].replace(regex=r'Woong, Hui Min Rachel', value='Woong, Rachel Hui Min')

df['NAME'] = df['NAME'].replace(regex=r'FENG, HAN LIN', value='Feng, Han Lin')
df['NAME'] = df['NAME'].replace(regex=r'Angela Tan Shi Jin', value='TAN, ANGELA')
df['NAME'] = df['NAME'].replace(regex=r'Zhang, Ruiheng, Zayden', value='Zhang, Zayden')



df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie El Redha Ang El Redha Ang', value='Jamie El-Red Ang')
df['NAME'] = df['NAME'].replace(regex=r'Jamie Norhisham', value='Jamie El-Red Ang')
df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie', value='Jamie El-Red Ang')

### 2024###

df['NAME'] = df['NAME'].replace(regex=r'D/O JAIGANTH, LAAVINIA', value='JAIGANTH, LAAVINIA')
df['NAME'] = df['NAME'].replace(regex=r'Jaiganth, Laavinia', value='JAIGANTH, LAAVINIA')
df['NAME'] = df['NAME'].replace(regex=r'Laavina Jaganith', value='JAIGANTH, LAAVINIA')


df['NAME'] = df['NAME'].replace(regex=r'Faith Zhen Ford', value='FAITH ZHEN FORD')
df['NAME'] = df['NAME'].replace(regex=r'Ford, Faith', value='FAITH ZHEN FORD')
df['NAME'] = df['NAME'].replace(regex=r'FORD, FAITH', value='FAITH ZHEN FORD')
df['NAME'] = df['NAME'].replace(regex=r'Ford Faith', value='FAITH ZHEN FORD')

df['NAME'] = df['NAME'].replace(regex=r'Koh Shun Yi Audrey', value='Koh, Shun Yi Audrey')
df['NAME'] = df['NAME'].replace(regex=r'Koh Shun Yi Audrey Q', value='Koh, Shun Yi Audrey')
df['NAME'] = df['NAME'].replace(regex=r'Shun Yi Audrey Koh', value='Koh, Shun Yi Audrey')
df['NAME'] = df['NAME'].replace(regex=r'KOH SHUN YI AUDREY', value='Koh, Shun Yi Audrey')



df['NAME'] = df['NAME'].replace(regex=r'Romaine Soh', value='ROMAINE SOH RUI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Soh, Romaine', value='ROMAINE SOH RUI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Soh, Rui Min Romaine', value='ROMAINE SOH RUI MIN')

df['NAME'] = df['NAME'].replace(regex=r'khoo, Tyeisha rene misson', value='Khoo, Tyeisha')

df['NAME'] = df['NAME'].replace(regex=r'Faith Jia Yin Koh Q', value='Koh, Faith Jia Yin')
df['NAME'] = df['NAME'].replace(regex=r'FAITH JIA YIN KOH', value='Koh, Faith Jia Yin')
df['NAME'] = df['NAME'].replace(regex=r'Faith Jia Yin Koh', value='Koh, Faith Jia Yin')
df['NAME'] = df['NAME'].replace(regex=r'Koh, Faith', value='Koh, Faith Jia Yin')

df['NAME'] = df['NAME'].replace(regex=r'LIM, EIFFER YING QI', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'lim, eiffer', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'Eiffer, Lim Ying Qi', value='LIM YING QI, EIFFER')

df['NAME'] = df['NAME'].replace(regex=r'Chui Ling Goh', value='GOH, CHUI LING')

df['NAME'] = df['NAME'].replace(regex=r'Chong Le Xuan, Chloe', value='CHONG LE XUAN, CHLOE')
df['NAME'] = df['NAME'].replace(regex=r'CHONG, LE XUAN, CHLOE', value='CHONG LE XUAN, CHLOE')
df['NAME'] = df['NAME'].replace(regex=r'CHONG, CHLOE LE XUAN', value='CHONG LE XUAN, CHLOE')



df['NAME'] = df['NAME'].replace(regex=r'SIDHOM, MANUELA', value='Sidhom, Manuela')
df['NAME'] = df['NAME'].replace(regex=r'Manuela Sidhom Q', value='Sidhom, Manuela')
df['NAME'] = df['NAME'].replace(regex=r'Manuela Sidhom q', value='Sidhom, Manuela')
df['NAME'] = df['NAME'].replace(regex=r'Manuela Sidhom', value='Sidhom, Manuela')

df['NAME'] = df['NAME'].replace(regex=r'^Gan,\sSteffi$', value='Gan, Steffi Kai Lin')
df['NAME'] = df['NAME'].replace(regex=r'Gan Kai-lin, Steffi', value='Gan, Steffi Kai Lin')




df['NAME'] = df['NAME'].replace(regex=r'Yu En Bernice, Tan', value='Tan Yu En, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Yu En Bernice', value='Tan Yu En, Bernice')
#df['NAME'] = df['NAME'].replace(regex=r'Tan, Bernice', value='Tan Yu En, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'Tan Yu En Bernice q', value='Tan Yu En, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'Tan Yu En Bernice Q', value='Tan Yu En, Bernice')
#df['NAME'] = np.where(((df['NAME']=='Tan, Bernice')&(df['DOB']=='2010-10-05')), 'Tan Yu En, Bernice', 'Tan, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'TAN, YUN RU BERNICE', value='Tan Yun Ru, Bernice')

#df['NAME'] = np.where(((df['NAME']=='Tan, Bernice')&(df['DOB']=='2000-11-16')), 'Tan Yun Ru, Bernice', 'Tan, Bernice')

df['NAME'] = df['NAME'].replace(regex=r'Ng, Yew Cheo', value='NG, YEW CHEO')

df['NAME'] = df['NAME'].replace(regex=r'Caelyn Chew Kai Ling Q', value='Chew Kai Ling, Caelyn')
df['NAME'] = df['NAME'].replace(regex=r'Caelyn Chew Kai Ling', value='Chew Kai Ling, Caelyn')

df['NAME'] = df['NAME'].replace(regex=r'FOO, NATALIE ZHI NING', value='Foo Zhi Ning, Natalie')
df['NAME'] = df['NAME'].replace(regex=r'Foo, Natalie', value='Foo Zhi Ning, Natalie')


df['NAME'] = df['NAME'].replace(regex=r'Xiu Yi, Leong', value='Leong, Xiu Yi')


df['NAME'] = df['NAME'].replace(regex=r'Shing Ling Goh', value='GOH SHING LING')
df['NAME'] = df['NAME'].replace(regex=r'Goh, Shing Ling', value='GOH SHING LING')



df['NAME'] = df['NAME'].replace(regex=r'Tia Louise Rozario', value='ROZARIO, TIA LOUISE')
df['NAME'] = df['NAME'].replace(regex=r'Rozario, Tia Louise', value='ROZARIO, TIA LOUISE')


df['NAME'] = df['NAME'].replace(regex=r'Chloe Chee En-Ya', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'En-ya, Chloe', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'Chee Chee En-ya, Chloe', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'Chloe Chee En-Ya Q', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'En-ya, Chloe', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'Chee En-ya, Chloe', value='CHLOE CHEE EN-YA')


df['NAME'] = df['NAME'].replace(regex=r'Chuhan Zhong', value='Zhong, Chuhan')
df['NAME'] = df['NAME'].replace(regex=r'Megan Anne Ying Ka Mun Q', value='Zhong,')


df['NAME'] = df['NAME'].replace(regex=r'Lim Yee Chern Clara Q', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'LIM, YEE CHERN CLARA', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'Clara Lim Yee Chern', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Clara', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'Lim Yee Chern Clara', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'YEE CHERN CLARA LIM', value='LIM YEE CHERN CLARA')


df['NAME'] = df['NAME'].replace(regex=r'CHEW, JADE', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'Jade, Chew Yiu Tse', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'Chew, Jade', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'CHEW, YIU TSE JADE', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'Chew, Yiu Tse Jade', value='CHEW YIU TSE JADE')

df['NAME'] = df['NAME'].replace(regex=r'Tay, Esther SHEE WEI', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'Esther Tay', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'TAY, ESTHER SHEE WEI', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'TAY, ESTHER', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'Tay, Esther', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'ESTHER TAY SHEE WEI', value='TAY, ESTHER')



df['NAME'] = df['NAME'].replace(regex=r'Han Qi Feng', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'FENG HAN QI', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'Han Qi Feng', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'Feng Han Qi', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'Feng, Han Qi', value='FENG, HAN QI')

df['NAME'] = df['NAME'].replace(regex=r'Syed Ahmed Riadh, Sharifah Falisha', value='SYED AHMED RIADH, SHARIFAH FALISHA')
df['NAME'] = df['NAME'].replace(regex=r'Sharifah Falisha Binte Syed Ahmed', value='SYED AHMED RIADH, SHARIFAH FALISHA')

df['NAME'] = df['NAME'].replace(regex=r'Sam, Vanessa', value='SAM, VANESSA')

df['NAME'] = df['NAME'].replace(regex=r'Vanessa Lee Ying Zhuang', value='LEE, VANESSA')
df['NAME'] = df['NAME'].replace(regex=r'Lee Ying Zhuang, Vanessa', value='LEE, VANESSA')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Vanessa', value='LEE, VANESSA')


df['NAME'] = df['NAME'].replace(regex=r'SNG, MICHELLE', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'SNG, SUAT LI, MICHELLE', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'Michelle Sng Suat Li', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'Sng, Michelle', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'Michelle Sng Suat Li', value='MICHELLE SNG')

df['NAME'] = df['NAME'].replace(regex=r'Rachel See Boon Lay', value='Rachel See')

df['NAME'] = df['NAME'].replace(regex=r'Yee, Wai Teng Melissa', value='YEE, MELISSA')
df['NAME'] = df['NAME'].replace(regex=r'YEE WAI TENG, MELISSA', value='YEE, MELISSA')

df['NAME'] = df['NAME'].replace(regex=r'Kerstin Ong Jing Rong', value='ONG, KERSTIN')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Kerstin', value='ONG, KERSTIN')


df['NAME'] = df['NAME'].replace(regex=r'Chua, Joshua', value='CHUA, JOSHUA')

df['NAME'] = df['NAME'].replace(regex=r'Raphael, Ryan', value='RAPHAEL, RYAN')
df['NAME'] = df['NAME'].replace(regex=r'NG, RYAN RAPHAEL', value='RAPHAEL, RYAN')

df['NAME'] = df['NAME'].replace(regex=r'Xander Ho Ann Heng', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'Ho, Xander Ann Heng', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'HO, XANDER, ANN HENG', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'XANDER, HO ANN HENG', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'Ho Ann Heng, Xander', value='HO ANN HENG, XANDER')

df['NAME'] = df['NAME'].replace(regex=r'Tan, Tate', value='TATE TAN FUNG')

df['NAME'] = df['NAME'].replace(regex=r'Tan Hong An, Daryl', value='TAN, DARYL')
df['NAME'] = df['NAME'].replace(regex=r'Tan Daryl', value='TAN, DARYL')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Daryl', value='TAN, DARYL')

df['NAME'] = df['NAME'].replace(regex=r'Ong Ying Tat', value='ONG, YING TAT')
df['NAME'] = df['NAME'].replace(regex=r'Ong Ying Tat Q', value='ONG, YING TAT')

df['NAME'] = df['NAME'].replace(regex=r'S/O SUBASH SOMAN, PRAHARSH RYAN', value='RYAN, PRAHARSH')
df['NAME'] = df['NAME'].replace(regex=r'Ryan, Praharsh', value='RYAN, PRAHARSH')
df['NAME'] = df['NAME'].replace(regex=r'Ryan Praharsh', value='RYAN, PRAHARSH')
df['NAME'] = df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='RYAN, PRAHARSH')

df['NAME'] = df['NAME'].replace(regex=r'LIM, CHUN KHAI, JON', value='LIM, JON')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Jon', value='LIM, JON')

df['NAME'] = df['NAME'].replace(regex=r'Wei Guan, Chong', value='CHONG, WEI GUAN')
df['NAME'] = df['NAME'].replace(regex=r'Chong, Wei Guan', value='CHONG, WEI GUAN')


df['NAME'] = df['NAME'].replace(regex=r'Mohamed, Haja Fayiz', value='FAYIZ, MOHAMED HAJA')
df['NAME'] = df['NAME'].replace(regex=r'Osman, Amir Rusyaidi', value='OSMAN, AMIR RUSYAIDI')

df['NAME'] = df['NAME'].replace(regex=r'Jing Heng Ng', value='NG, JING HENG')
df['NAME'] = df['NAME'].replace(regex=r'Ng, Jing Heng', value='NG, JING HENG')


df['NAME'] = df['NAME'].replace(regex=r'Tan, Jonathan', value='TAN, JONATHAN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Jonathan Tan', value='TAN, JONATHAN')
df['NAME'] = df['NAME'].replace(regex=r'JONATHAN TAN', value='TAN, JONATHAN')



df['NAME'] = df['NAME'].replace(regex=r'Lucas Le Cong Fun', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Fun Le Cong, Lucas', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Fun Le Cong, Lucas Q', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Fun, Le Cong Lucas', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'FUN, LE CONG LUCAS', value='FUN LE CONG LUCAS')


df['NAME'] = df['NAME'].replace(regex=r'PERCY MUNCHERJI, ZUBIN', value='ZUBIN PERCY MUNCHERJI')
df['NAME'] = df['NAME'].replace(regex=r'Muncherji, Zubin', value='ZUBIN PERCY MUNCHERJI')
df['NAME'] = df['NAME'].replace(regex=r'MUNCHERJI, ZUBIN PERCY', value='ZUBIN PERCY MUNCHERJI')
df['NAME'] = df['NAME'].replace(regex=r'Zubin Percy Muncherji', value='ZUBIN PERCY MUNCHERJI')

df['NAME'] = df['NAME'].replace(regex=r'Quek, Jun Jie Calvin', value='QUEK, JUN JIE CALVIN')
df['NAME'] = df['NAME'].replace(regex=r'Jun Jie Calvin Quek', value='QUEK, JUN JIE CALVIN')

df['NAME'] = df['NAME'].replace(regex=r'Chia, Shawn', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'CHIA, WEI EN, SHAWN', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'Chia Wei En, Shawn', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'Wei En Shawn, Chia', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'Shawn Chia Wei En', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'CHIA, SHAWN WEI EN', value='SHAWN CHIA WEI EN')


df['NAME'] = df['NAME'].replace(regex=r'Jie Cong Jayden, Tan', value='JIE CONG JAYDEN, TAN')
df['NAME'] = df['NAME'].replace(regex=r'TAN , JIE CONG, JAYDEN', value='JIE CONG JAYDEN, TAN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Jayden', value='JIE CONG JAYDEN, TAN')

df['NAME'] = df['NAME'].replace(regex=r'Lim, Oliver', value='LIM, OLIVER')

df['NAME'] = df['NAME'].replace(regex=r'YEE, ERIC CHUN WAI', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'YEE CHUN WAI, ERIC', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Eric Yee Chun Wai', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee Chun Wai, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee, Chun Wai, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee Chun Wai, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'YEE, ERIC CHUN WAI', value='YEE, CHUN WAI, ERIC')


df['NAME'] = df['NAME'].replace(regex=r'Rui Yong Soh', value='SOH RUI YONG GUILLAUME')

df['NAME'] = df['NAME'].replace(regex=r'TAN, AARON JUSTIN', value='TAN, AARON JUSTIN WEN JIE')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Aaron Justin', value='TAN, AARON JUSTIN WEN JIE')


df['NAME'] = df['NAME'].replace(regex=r'Shaun, Goh', value='GOH, SHAUN')
df['NAME'] = df['NAME'].replace(regex=r'Goh, Shaun', value='GOH, SHAUN')
df['NAME'] = df['NAME'].replace(regex=r'Shaun Goh', value='GOH, SHAUN')

df['NAME'] = df['NAME'].replace(regex=r'Ethan Yan', value='YAN, ETHAN')

df['NAME'] = df['NAME'].replace(regex=r'Brandon Heng Fu Hong', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'Heng, Brandon', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'Brandon Heng Fu Hong', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'HENG, BRANDON', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'HENG, FU HONG, BRANDON', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'HENG, FU HONG BRANDON', value='HENG, BRANDON')

df['NAME'] = df['NAME'].replace(regex=r'Tung Hon Andrew Pak', value='PAK, ANDREW')
df['NAME'] = df['NAME'].replace(regex=r'Pak, Andrew', value='PAK, ANDREW')

df['NAME'] = df['NAME'].replace(regex=r'Wong Yijie Lucas', value='WONG YIJIE LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Wong, Yijie Lucas', value='WONG YIJIE LUCAS')

df['NAME'] = df['NAME'].replace(regex=r'Mun Jern Wei', value='MUN, IVAN')
df['NAME'] = df['NAME'].replace(regex=r'Mun, Jern Wei Ivan', value='MUN, IVAN')
df['NAME'] = df['NAME'].replace(regex=r'Mun, Ivan', value='MUN, IVAN')


df['NAME'] = df['NAME'].replace(regex=r'Conrad Kangli Emery', value='EMERY, CONRAD')
df['NAME'] = df['NAME'].replace(regex=r'Conrad, Kangli Emery', value='EMERY, CONRAD')
df['NAME'] = df['NAME'].replace(regex=r'Kangli Emery Conrad', value='EMERY, CONRAD')
df['NAME'] = df['NAME'].replace(regex=r'Conrad Kangli Emery Q', value='EMERY, CONRAD')

df['NAME'] = df['NAME'].replace(regex=r'Melvin, Wong', value='WONG, MELVIN')
df['NAME'] = df['NAME'].replace(regex=r'MELVIN, WONG', value='WONG, MELVIN')

df['NAME'] = df['NAME'].replace(regex=r'Caleb Hia', value='HIA, CALEB')

df['NAME'] = df['NAME'].replace(regex=r'Jun Yu Low', value='LOW, JUN YU')
df['NAME'] = df['NAME'].replace(regex=r'Low Jun Yu', value='LOW, JUN YU')
df['NAME'] = df['NAME'].replace(regex=r'Low, Jun Yu', value='LOW, JUN YU')

df['NAME'] = df['NAME'].replace(regex=r'Gabriel Lee', value='LEE GABRIEL JIN YI')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Gabriel', value='LEE GABRIEL JIN YI')

df['NAME'] = df['NAME'].replace(regex=r'Andrew George Medina', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'Medina, Andrew', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'Medina, Andrew George', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'George Medina, Andrew', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'George George Medina, Andrew', value='MEDINA, ANDREW GEORGE')



df['NAME'] = df['NAME'].replace(regex=r'^Harry\sIrfan\sCurra$', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Curran, Harry Irfann', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'^Curran,\sHarry$', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Harry Irfan Curran Q', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Harry Irfan Curran', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Curran, Harry Irfan', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'CURRAN, HARRY IRFAN', value='HARRY IRFAN CURRAN')


df['NAME'] = df['NAME'].replace(regex=r'Wei Jun Huang', value='HUANG WEI JUN')
df['NAME'] = df['NAME'].replace(regex=r'HUANG, WEI JUN', value='HUANG WEI JUN')

df['NAME'] = df['NAME'].replace(regex=r'., Hariharan', value=', HARIHARAN')
df['NAME'] = df['NAME'].replace(regex=r'S/O KrishnS/O Krishna, Hariharan', value=', HARIHARAN')
df['NAME'] = df['NAME'].replace(regex=r'KRISHNAN, HARIHARAN S/O', value=', HARIHARAN')

'''

"\n\n# Correct some of the known name variations\n# Need to fix 2024 NSG names extracts (extracting Q/q with name)\n\ndf['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)\ndf['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)\ndf['NAME'] = df['NAME'].str.replace('\r', '', regex=True)\ndf['NAME'] = df['NAME'].str.replace('\n', '', regex=True)\ndf['NAME'] = df['NAME'].str.strip()\n\n\n\ndf['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie El Redha Ang El Redha Ang', value='Norhisham, Jamie El Redha Ang')\n\n\n\ndf['NAME'] = df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')\ndf['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')\ndf['NAME'] = df['NAME'].replace(regex=r'TAN, SHEE RU, ELIZABETH-A', value='TAN, ELIZABETH-ANN')\ndf['NAME'] = df['NAME'].replace(regex=r'Elizabeth-Ann Tan', value='TAN, ELIZABETH-ANN')\ndf['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth-Ann', value='TAN, ELIZAB

In [1598]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,458.0,458.00,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,15.03,15.03,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,22.43,22.43,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,...,192.5760,195.4080,198.240,252.1,252.10,-59.5240,-56.6920,-53.860,-63.30,-28.527542
20201,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,...,45.4716,46.1403,46.809,59.79,59.79,-14.3184,-13.6497,-12.981,-15.21,-29.118439
20202,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,...,40.1472,40.7376,41.328,46.33,46.33,-6.1828,-5.5924,-5.002,-6.97,-12.708333
20203,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,...,45.4716,46.1403,46.809,62.1,62.10,-16.6284,-15.9597,-15.291,-17.52,-34.300135


In [1599]:

# Read a variation name list and corrections from CSVs
'''
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

names = pd.read_csv("name_variations.csv")

for index, row in names.iterrows():
        
    print(names.VARIATION, names.NAME)
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")
'''

'\nos.chdir(\'/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/\')\n\nnames = pd.read_csv("name_variations.csv")\n\nfor index, row in names.iterrows():\n        \n    print(names.VARIATION, names.NAME)\n    df[\'NAME\'] = df[\'NAME\'].replace(regex=rf"{row[\'VARIATION\']}", value=f"{row[\'NAME\']}")\n'

In [1600]:
# Read name variations from GCS name lists bucket (Still in beta)


df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})

# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")


In [1601]:
df_names=df[((df['NAME']=='Heng, Richard')|(df['NAME']=='Marican, Shohib'))]

In [1602]:
df_names.to_csv("names.csv", encoding='utf-8')

In [1603]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

#df_select = df[(df['TEAM']!='Malaysia') & (df['TEAM']!='THAILAND') & (df['TEAM']!='China') & (df['TEAM']!='South Korea') & (df['TEAM']!='Laos') & (df['TEAM']!='Philippines') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Chinese Taipei') & (df['TEAM']!='Gurkha Contingent') & (df['TEAM']!='Australia') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Hong Kong') & (df['TEAM']!='PERAK')] 

df_select = df[(df['TEAM']!='Malaysia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='China') 
                       &(df['TEAM']!='South Korea')&(df['TEAM']!='Laos') 
                       &(df['TEAM']!='Philippines')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Chinese Taipei')&(df['TEAM']!='Gurkha Contingent') 
                       &(df['TEAM']!='Australia')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Hong Kong')&(df['TEAM']!='PERAK')&(df['TEAM']!='Sri Lanka') 
                       &(df['TEAM']!='Indonesia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='MALAYSIA') 
                       &(df['TEAM']!='PHILIPPINES') & (df['TEAM']!='SOUTH KOREA')&(df['TEAM']!='Waseda') 
                       &(df['TEAM']!='LAOS')&(df['TEAM']!='CHINESE TAIPEI')
                       &(df['TEAM']!='INDIA')&(df['TEAM']!='Hong Kong, China')&(df['TEAM']!='AIC JAPAN')] 

In [1604]:
df_select

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,KOH WEI SHIEN,4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,458.0,458.00,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,15.03,15.03,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,22.43,22.43,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,...,192.5760,195.4080,198.240,252.1,252.10,-59.5240,-56.6920,-53.860,-63.30,-28.527542
20201,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,...,45.4716,46.1403,46.809,59.79,59.79,-14.3184,-13.6497,-12.981,-15.21,-29.118439
20202,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,...,40.1472,40.7376,41.328,46.33,46.33,-6.1828,-5.5924,-5.002,-6.97,-12.708333
20203,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,...,45.4716,46.1403,46.809,62.1,62.10,-16.6284,-15.9597,-15.291,-17.52,-34.300135


In [1605]:
df_select[df_select['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
726,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,...,NaN,NaN,NaN,299.3,299.30,NaN,NaN,NaN,NaN,NaN
728,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,...,1923.210,1951.4925,1979.775,1871.4,1871.40,51.810,80.0925,108.375,14.1,5.747812
729,SOH RUI YONG GUILLAUME,15:07.7,INDIVIDUAL,33,46,None,5000m,None,None,None,...,901.068,914.3190,927.570,907.7,907.70,-6.632,6.6190,19.870,-24.3,2.249264
730,SOH RUI YONG GUILLAUME,08:35.8,INDIVIDUAL,33,None,None,3000m,None,None,None,...,NaN,NaN,NaN,515.8,515.80,NaN,NaN,NaN,NaN,NaN
1560,SOH RUI YONG GUILLAUME,15:08,None,None,46,None,"5km, Road",None,None,None,...,NaN,NaN,NaN,3.45,3.45,NaN,NaN,NaN,NaN,NaN
1561,SOH RUI YONG GUILLAUME,08:35.8,None,None,12,None,3000m,None,None,None,...,NaN,NaN,NaN,515.8,515.80,NaN,NaN,NaN,NaN,NaN
1562,SOH RUI YONG GUILLAUME,32:03:00,None,None,2,None,"10km, Road",None,None,None,...,NaN,NaN,NaN,115380.0,115380.00,NaN,NaN,NaN,NaN,NaN
1563,SOH RUI YONG GUILLAUME,15:06.1,None,None,12,None,5000m,None,None,None,...,901.068,914.3190,927.570,906.1,906.10,-5.032,8.2190,21.470,-22.7,2.430383
1599,SOH RUI YONG GUILLAUME,1:09:28,None,None,38,None,Half Marathon,None,None,None,...,NaN,NaN,NaN,4168.0,4168.00,NaN,NaN,NaN,NaN,NaN
1693,SOH RUI YONG GUILLAUME,2:24:07,None,None,41,None,Marathon,None,None,None,...,9535.980,9676.2150,9816.450,8647.0,8647.00,888.980,1029.2150,1169.450,702.0,12.508824


In [1606]:
df_select[(df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m')]

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
9681,"Tan, Kian Ye",29.03,HENRY PARK PRIMARY SCHOOL,11,1,U12,Dash,200,,nan,...,21.4404,21.7557,22.071,29.03,29.03,-7.5896,-7.2743,-6.959,-8.01,-33.106565


In [1607]:
# Remove spurious SEED results 

#df_select=df_select[~((df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m'))]
#df_select=df_select[~((df_select['NAME']=='LOH, BEAVERLY') & (df_select['MAPPED_EVENT']=='400m'))]
#df_select=df_select[~((df_select['NAME']=='Vaz, Luisa Marie') & (df_select['MAPPED_EVENT']=='High jump'))]


In [1608]:
df_select

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,KOH WEI SHIEN,4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,458.0,458.00,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,15.03,15.03,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,22.43,22.43,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,...,192.5760,195.4080,198.240,252.1,252.10,-59.5240,-56.6920,-53.860,-63.30,-28.527542
20201,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,...,45.4716,46.1403,46.809,59.79,59.79,-14.3184,-13.6497,-12.981,-15.21,-29.118439
20202,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,...,40.1472,40.7376,41.328,46.33,46.33,-6.1828,-5.5924,-5.002,-6.97,-12.708333
20203,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,...,45.4716,46.1403,46.809,62.1,62.10,-16.6284,-15.9597,-15.291,-17.52,-34.300135


In [1609]:
'''
# Read list of foreigners

foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')
'''


"\n# Read list of foreigners\n\nforeigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')\n"

In [1610]:
# Read list of foreigners from GCS bucket

file_path = "gs://name_lists/List of Foreigners.csv"
foreigners = pd.read_csv(file_path,
                 sep=",",
                 encoding="unicode escape",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


In [1611]:
foreigners

,LAST_NAME,FIRST_NAME
0,Aaryan,Greuter Christoph
1,Akahodani,Takayuki
2,Apondar,Audric
3,Brooks,Ruby
4,Brouwer,Cees
...,...,...
236,ISMAIL,MUHAMMAD ZULFIQAR
237,Jayaganeson,Kirtisha
238,LIN,Yu Sian
239,King,Leonard


In [1612]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()

foreign_list = for1+for2+for3+for4 

foreign_list_casefold=[s.casefold() for s in foreign_list]

exclusions = foreign_list_casefold

no_foreigners_list = df_select.loc[~df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [1613]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [1614]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [1615]:
no_foreigners_list

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,KOH WEI SHIEN,4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,458.0,458.00,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,15.03,15.03,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,22.43,22.43,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200,"{'76': 'Bhuvanesh, S/O Gothandaraman', '81': '...",04:12.1,Singapore University of Social,nan,10,Open,Relay,1600,,,...,192.5760,195.4080,198.240,252.1,252.10,-59.5240,-56.6920,-53.860,-63.30,-28.527542
20201,"{'84': 'Chung, Laurene', '86': 'Koh, Li Shuen'...",59.79,Singapore University of Social,nan,11,Open,Relay,400,,,...,45.4716,46.1403,46.809,59.79,59.79,-14.3184,-13.6497,-12.981,-15.21,-29.118439
20202,"{'77': 'Abdul Hakim Bin Abdu, Hakim', '79': 'G...",46.33,Singapore University of Social,nan,8,Open,Relay,400,,,...,40.1472,40.7376,41.328,46.33,46.33,-6.1828,-5.5924,-5.002,-6.97,-12.708333
20203,"{'169': 'Nakamura, Yuna', '171': 'Matsui, Yuri...",01:02.1,Waseda Shibuya Senior High Sch,nan,7,Open,Relay,400,,,...,45.4716,46.1403,46.809,62.1,62.10,-16.6284,-15.9597,-15.291,-17.52,-34.300135


In [1616]:
# Choose the best performing event for each athlete

#top_performers_clean = excluded_list.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [1617]:
# Choose the best result for each event participated by every athlete

top_performers_clean = no_foreigners_list.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [1618]:
top_performers_clean.reset_index(inplace=True)


In [1619]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


top_performers_clean.to_csv('top_performers_clean_prod.csv', encoding='utf-8')

In [1620]:
top_performers_clean

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,18272,"yeo, husni",10.09m,Singapore Polytechnic,19,17,Open,Triple Jump,0,,...,15.3860,15.1505,14.915,10.09,10.09,-5.2960,-5.0605,-4.825,-5.61,-30.732484
1,17346,"ooi, chen si",9.22m,Ngee Ann Polytechnic,18,9,Open,Triple Jump,0,,...,13.1908,12.9889,12.787,9.22,9.22,-3.9708,-3.7689,-3.567,-4.24,-26.500743
2,15839,"chua, keira",10.16m,Cedar Girls Secondary School,16,5,U18,Triple Jump,0,,...,13.1908,12.9889,12.787,10.16,10.16,-3.0308,-2.8289,-2.627,-3.30,-19.517088
3,1720,"ZHONG, CHUHAN",12.26,None,None,None,None,Triple Jump,None,None,...,13.1908,12.9889,12.787,12.26,12.26,-0.9308,-0.7289,-0.527,-1.20,-3.915305
4,5414,Yvette Lee,10.13,VJC,None,7,A,Triple Jump,None,None,...,13.1908,12.9889,12.787,10.13,10.13,-3.0608,-2.8589,-2.657,-3.33,-19.739970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12028,8453,"., Mohd Farrel",31.28,Seng Kang Primary School,10,38,U11,Dash,150,,...,NaN,NaN,NaN,31.28,31.28,NaN,NaN,NaN,NaN,NaN
12029,13478,"., Dharkshitha",12:46.5,Cedar Girls Secondary School,14,5,U15,Race Walk,1500,,...,NaN,NaN,NaN,766.5,766.50,NaN,NaN,NaN,NaN,NaN
12030,12173,"., Cayden",06:05.1,North Vista,14,54,Open,Mile Run,1,,...,NaN,NaN,NaN,365.1,365.10,NaN,NaN,NaN,NaN,NaN
12031,6992,"., Asher",11.76,Erovra Club,6,16,7-Aug,Dash,60,,...,NaN,NaN,NaN,11.76,11.76,NaN,NaN,NaN,NaN,NaN


In [1621]:
'''
# Join 2024 best results for each event for each athlete

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/2023/')


df_yoy = pd.read_csv("Best_results_2023.csv")

df_yoy['NAME'] = df_yoy['NAME'].str.replace('\xa0', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\r', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\n', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.strip()
'''

'\n# Join 2024 best results for each event for each athlete\n\nos.chdir(\'/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/2023/\')\n\n\ndf_yoy = pd.read_csv("Best_results_2023.csv")\n\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'\xa0\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'[\x00-\x1f\x7f-\x9f]\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'\r\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'\n\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.strip()\n'

In [1622]:
#df_yoy

In [1623]:
'''

# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

yoy_performance = pd.merge(
    left=df_yoy, 
    right=top_performers_clean,
    how='left',
    left_on=['EVENT', 'GENDER', 'NAME'],
    right_on=['MAPPED_EVENT', 'GENDER', 'NAME'],
)

'''

"\n\n# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER\n\nyoy_performance = pd.merge(\n    left=df_yoy, \n    right=top_performers_clean,\n    how='left',\n    left_on=['EVENT', 'GENDER', 'NAME'],\n    right_on=['MAPPED_EVENT', 'GENDER', 'NAME'],\n)\n\n"

In [1624]:
#yoy_performance

In [1625]:
'''

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


yoy_performance.to_csv('yoy_performance_prod.csv', index=False, encoding='utf-8')

'''

"\n\nos.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')\n\n\nyoy_performance.to_csv('yoy_performance_prod.csv', index=False, encoding='utf-8')\n\n"

In [1626]:
# Choose best performance for each event

#tiered_performers = top_performers_clean.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

tiered_performers = top_performers_clean


In [1627]:
tiered_performers

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,18272,"yeo, husni",10.09m,Singapore Polytechnic,19,17,Open,Triple Jump,0,,...,15.3860,15.1505,14.915,10.09,10.09,-5.2960,-5.0605,-4.825,-5.61,-30.732484
1,17346,"ooi, chen si",9.22m,Ngee Ann Polytechnic,18,9,Open,Triple Jump,0,,...,13.1908,12.9889,12.787,9.22,9.22,-3.9708,-3.7689,-3.567,-4.24,-26.500743
2,15839,"chua, keira",10.16m,Cedar Girls Secondary School,16,5,U18,Triple Jump,0,,...,13.1908,12.9889,12.787,10.16,10.16,-3.0308,-2.8289,-2.627,-3.30,-19.517088
3,1720,"ZHONG, CHUHAN",12.26,None,None,None,None,Triple Jump,None,None,...,13.1908,12.9889,12.787,12.26,12.26,-0.9308,-0.7289,-0.527,-1.20,-3.915305
4,5414,Yvette Lee,10.13,VJC,None,7,A,Triple Jump,None,None,...,13.1908,12.9889,12.787,10.13,10.13,-3.0608,-2.8589,-2.657,-3.33,-19.739970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12028,8453,"., Mohd Farrel",31.28,Seng Kang Primary School,10,38,U11,Dash,150,,...,NaN,NaN,NaN,31.28,31.28,NaN,NaN,NaN,NaN,NaN
12029,13478,"., Dharkshitha",12:46.5,Cedar Girls Secondary School,14,5,U15,Race Walk,1500,,...,NaN,NaN,NaN,766.5,766.50,NaN,NaN,NaN,NaN,NaN
12030,12173,"., Cayden",06:05.1,North Vista,14,54,Open,Mile Run,1,,...,NaN,NaN,NaN,365.1,365.10,NaN,NaN,NaN,NaN,NaN
12031,6992,"., Asher",11.76,Erovra Club,6,16,7-Aug,Dash,60,,...,NaN,NaN,NaN,11.76,11.76,NaN,NaN,NaN,NaN,NaN


In [1628]:
# Identify Tier 1/2/3 performers

#top_performers_clean['TIER'] = np.where((top_performers_clean['Delta_Benchmark']>=0), 'Tier 1',    
#                                np.where(((top_performers_clean['Delta_Benchmark']<0) & (top_performers_clean['Delta2']>=0)), 'Tier2',
#                                np.where(((top_performers_clean['Delta2']<0) & (top_performers_clean['Delta3.5']>=0)), 'Tier3', ' ')))


tiered_performers['TIER'] = np.where((tiered_performers['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((tiered_performers['Delta_Benchmark']<0) & (tiered_performers['Delta2']>=0)), 'Tier 2',
                                np.where(((tiered_performers['Delta2']<0) & (tiered_performers['Delta3.5']>=0)), 'Tier 3', 
                                np.where(((tiered_performers['Delta3.5']<0) & (tiered_performers['Delta5']>=0)), 'Tier 4', ' '))))



In [1629]:
# Apply Rule e, two per tier rule



In [1630]:
tiered_performers[tiered_performers['MAPPED_EVENT']=='Marathon']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
513,741,YVONNE ELIZABETH CHEE,3:12:19,INDIVIDUAL,45,None,None,Marathon,None,None,...,10584.945,10738.35,11539.0,11539.0,-1107.46,-954.055,-800.65,-1312.0,-7.828787,
514,774,YIK SIONG WONG,2:46:37,INDIVIDUAL,29,None,None,Marathon,None,None,...,9676.215,9816.45,9997.0,9997.0,-461.02,-320.785,-180.55,-648.0,-1.931223,
515,751,YEW MENG TAN,2:55:18,INDIVIDUAL,41,None,None,Marathon,None,None,...,9676.215,9816.45,10518.0,10518.0,-982.02,-841.785,-701.55,-1169.0,-7.504011,
516,750,YAPING MA,2:51:39,INDIVIDUAL,41,None,None,Marathon,None,None,...,9676.215,9816.45,10299.0,10299.0,-763.02,-622.785,-482.55,-950.0,-5.161515,
517,760,XIUYING HU,3:10:29,INDIVIDUAL,37,None,None,Marathon,None,None,...,9676.215,9816.45,11429.0,11429.0,-1893.02,-1752.785,-1612.55,-2080.0,-17.248369,
518,766,XIAO WEN GIEBERT FOO,2:48:03,INDIVIDUAL,33,None,None,Marathon,None,None,...,9676.215,9816.45,10083.0,10083.0,-547.02,-406.785,-266.55,-734.0,-2.851107,
519,1600,Wei Xiang Gordon Lim,2:36:19,None,None,566,None,Marathon,None,None,...,9676.215,9816.45,9379.0,9379.0,156.98,297.215,437.45,-30.0,4.679110,Tier 2
520,748,"WONG, MELVIN",2:46:31,INDIVIDUAL,41,None,None,Marathon,None,None,...,9676.215,9816.45,9991.0,9991.0,-455.02,-314.785,-174.55,-642.0,-1.867045,
521,753,"WONG, KEN MUN",2:48:35,INDIVIDUAL,38,None,None,Marathon,None,None,...,9676.215,9816.45,10115.0,10115.0,-579.02,-438.785,-298.55,-766.0,-3.193390,
522,765,WEI JIE HIA,2:43:57,INDIVIDUAL,34,None,None,Marathon,None,None,...,9676.215,9816.45,9837.0,9837.0,-301.02,-160.785,-20.55,-488.0,-0.219810,


In [1631]:
# Drop rows without a SEAG benchmark

final_df = tiered_performers[tiered_performers['BENCHMARK'].notna()]


In [1632]:
# Process dates to extract age

# Map NSG divisions into age

mask = (final_df['DIVISION'].str.contains(r'A', na=False))
final_df.loc[mask, 'AGE'] = '18.5'

mask = (final_df['DIVISION'].str.contains(r'B', na=False))
final_df.loc[mask, 'AGE'] = '16'

mask = (final_df['DIVISION'].str.contains(r'C', na=False))
final_df.loc[mask, 'AGE'] = '13.5'

mask = (final_df['DIVISION'].str.contains(r'O', na=False))
final_df.loc[mask, 'AGE'] = '12'



In [1633]:
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


final_df['DOB_new'] = final_df['DOB'].apply(length)



#B = parser.parse("10-09-2021", dayfirst = True)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/3934348418.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['DOB_new'] = final_df['DOB'].apply(length)


In [1634]:
final_df['DOB_new'] = pd.to_datetime(final_df['DOB_new'], errors='coerce')

final_df['year_extract']=final_df['DOB_new'].dt.strftime('%Y')

final_df['year_extract'] = pd.to_numeric(final_df['year_extract'])

final_df['age_extract'] = 2025 - final_df['year_extract']


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1594838337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['DOB_new'] = pd.to_datetime(final_df['DOB_new'], errors='coerce')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1594838337.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['year_extract']=final_df['DOB_new'].dt.strftime('%Y')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1594838337.py:5: SettingWithCopyWarning: 
A value is 

In [1635]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


final_df['age_extract']=final_df['age_extract'].apply(age)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1795310803.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_extract']=final_df['age_extract'].apply(age)


In [1636]:
# If NSG event then choose AGE otherwise choose age_extract

condition1 = final_df['COMPETITION']=='National School Games'
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/4177143290.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)


In [1637]:
# Change to numeric

final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1263148356.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)


In [1638]:
final_df[final_df['NAME']=='Marican, Shohib']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract
2924,1302,"Marican, Shohib",16:52.2,Macritchie Runners 25,12,8,Open,Run,5000,,...,1012.2,-111.132,-97.881,-84.63,-128.8,-9.580032,,1997-05-19,1997.0,28.0
4420,1301,"Marican, Shohib",11:43.8,Macritchie Runners 25,12,3,Open,Steeplechase,3000,(0.914m),...,703.8,-158.100,-150.075,-142.05,-168.8,-26.551402,,1997-05-19,1997.0,28.0
5785,1300,"Marican, Shohib",04:16.8,Macritchie Runners 25,12,8,Open,Run,1500,,...,256.8,-12.612,-9.021,-5.43,-17.4,-2.268170,,1997-05-19,1997.0,28.0


In [1639]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


final_df.to_csv('tiered_performers_prod.csv', encoding='utf-8')

In [1640]:
# Rank everyone without filters

all_ranking = final_df.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
all_ranking['Rank'] = all_ranking.groupby(['GENDER', 'MAPPED_EVENT']).cumcount() + 1


In [1641]:
all_ranking

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank
189,18725,LEE GABRIEL JIN YI,15.67m,NUS,12,1,Open,Triple Jump,0,,...,0.284,0.5195,0.755,-0.03,4.808917,Tier 2,2003-02-23,2003.0,22.0,1
138,1714,"MEDINA, ANDREW GEORGE",15.43,None,None,None,None,Triple Jump,None,None,...,0.044,0.2795,0.515,-0.27,3.280255,Tier 2,NaT,NaN,NaN,2
202,18850,"Kapil, Arnav",15.03m,WINGS ATHLETICS CLUB,12,1,Open,Triple Jump,0,,...,-0.356,-0.1205,0.115,-0.67,0.732484,Tier 4,2003-08-25,2003.0,22.0,3
57,1876,Tan Shou Yi Rei (Chen Shouyi),14.12,None,None,1,None,Triple Jump,None,None,...,-1.266,-1.0305,-0.795,-1.58,-5.063694,,NaT,NaN,NaN,4
194,16135,"Kumar, Viresh",13.92m,Unattached,12,2,Open,Triple Jump,0,,...,-1.466,-1.2305,-0.995,-1.78,-6.337580,,2007-03-01,2007.0,18.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7897,6736,"Chan, Ashley",46:42.8,National University Singapore,12,6,Open,Run,10000,,...,-629.180,-597.2150,-565.250,-671.80,-26.525106,,NaT,NaN,NaN,9
7879,6798,"Lee, Annette",47:04.3,Singapore Institute of Managem,12,7,Open,Run,10000,,...,-650.680,-618.7150,-586.750,-693.30,-27.534022,,NaT,NaN,NaN,10
7873,6885,"Loh, Cheyenne Xinyi",54:02.0,Singapore Management Universit,12,8,Open,Run,10000,,...,-1068.380,-1036.4150,-1004.450,-1111.00,-47.135148,,NaT,NaN,NaN,11
7884,605,"LEE, VANESSA",35:55:00,SINGAPORE,26,8,None,10000m,None,None,...,-127126.380,-127094.4150,-127062.450,-127169.00,-5962.573909,,1998-04-07,1998.0,27.0,12


In [1642]:
# Apply Rule E, two per tier only, therefore move 3rd and 4th placings down one tier

#all_ranking['TIER_ADJ'] = np.where(
#                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==3)), 'Tier 2',    
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==4)), 'Tier2',
#                                np.where(
#                                ((all_ranking['TIER']=='Tier2') & (all_ranking['Rank']==3)), 'Tier3', 
#                                np.where(
#                                ((all_ranking['TIER']=='Tier2') & (all_ranking['Rank']==4)), 'Tier3', ' ')
#                                
#                                )))


#all_ranking['TIER_ADJ'] = np.where(
#                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==3)), 'Tier 2',    
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==4)), 'Tier 2',
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==3)), 'Tier 3', 
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==4)), 'Tier 3', 
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==5)), 'Tier 3', 
#                                np.where(  
#                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==6)), 'Tier 3', 
#                                np.where(    
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==3)), 'Tier 4', 
#                                np.where(                             
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==4)), 'Tier 4', 
#                                np.where(                             
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==5)), 'Tier 4', 
#                                np.where(                             
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==6)), 'Tier 4', 
#                                np.where(                             
#                                
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==4)), 'Tier 4', all_ranking['TIER'])
#                                
#                               ))))))))))

all_ranking['TIER_ADJ'] = np.where(
                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==3)), 'Tier 2',    
                                np.where(
                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']>=4)), 'Tier 2',
                                np.where(
                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==3)), 'Tier 3', 
                                np.where(
                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']>=4)), 'Tier 3', 
                                np.where(                             
                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==3)), 'Tier 4', 
                                np.where(                             
                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']>=4)), 'Tier 4', all_ranking['TIER']) 
                
                                )))))



In [1643]:
all_ranking.to_csv('all_ranking.csv', encoding='utf-8')

In [1644]:
# Re-rank based on MAPPED_EVENT, GENDER, TIER_ADJ & PERF_SCALAR

rerank = all_ranking.sort_values(['MAPPED_EVENT','GENDER','TIER_ADJ', 'PERF_SCALAR'], ascending=[False, False, False, False])
rerank['Rank_ADJ'] = rerank.groupby(['MAPPED_EVENT', 'GENDER', 'TIER_ADJ']).cumcount() + 1


In [1645]:
rerank_filtered=rerank[rerank['TIER_ADJ']!=' ']

In [1646]:
rerank_filtered

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank,TIER_ADJ,Rank_ADJ
202,18850,"Kapil, Arnav",15.03m,WINGS ATHLETICS CLUB,12,1,Open,Triple Jump,0,,...,0.1150,-0.67,0.732484,Tier 4,2003-08-25,2003.0,22.0,3,Tier 4,1
189,18725,LEE GABRIEL JIN YI,15.67m,NUS,12,1,Open,Triple Jump,0,,...,0.7550,-0.03,4.808917,Tier 2,2003-02-23,2003.0,22.0,1,Tier 2,1
138,1714,"MEDINA, ANDREW GEORGE",15.43,None,None,None,None,Triple Jump,None,None,...,0.5150,-0.27,3.280255,Tier 2,NaT,NaN,NaN,2,Tier 2,2
102,1673,"ROZARIO, TIA LOUISE",13.01,None,None,1,None,Triple Jump,None,None,...,0.2230,-0.45,1.656761,Tier 3,2000-10-14,2000.0,25.0,1,Tier 3,1
379,1787,Anson Loh Ding Rong,17.2,None,None,1,None,Shot Put,None,5kg,...,0.7650,-0.10,4.421965,Tier 2,NaT,NaN,NaN,1,Tier 2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6470,18780,"TAN, ELIZABETH-ANN",11.99,SINGAPORE,12,1,Open,Dash,100,,...,0.3475,-0.24,2.957447,Tier 3,2003-09-23,2003.0,22.0,2,Tier 3,1
6631,1652,"PEREIRA, VERONICA SHANTI",11.47,None,None,1,None,100m,None,None,...,0.8675,0.28,7.382979,Tier 1,1996-09-20,1996.0,29.0,1,Tier 1,1
7883,6709,"LIM, JON",32:54.4,National University Singapore,12,1,Open,Run,10000,,...,5.3750,-88.90,0.285070,Tier 4,NaT,NaN,NaN,2,Tier 4,1
7866,728,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,...,108.3750,14.10,5.747812,Tier 1,1991-04-07,1991.0,34.0,1,Tier 1,1


In [1647]:
rerank_filtered.to_csv('rerank_filtered.csv', encoding='utf-8')

# Top 8 Performers

In [820]:
# Filter top 8 performances for each event and gender

top_eight_performers = tiered_performers.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'],ascending=[False, False, False]).groupby(['GENDER', 'MAPPED_EVENT']).head(8).reset_index(drop=True)

top_eight_performers

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
0,23028,LEE GABRIEL JIN YI,15.67m,NUS,21,1,Open,Triple Jump,0,,...,15.1505,14.915,15.67,15.67,0.284,0.5195,0.755,-0.03,4.808917,Tier2
1,5676,"MEDINA, ANDREW GEORGE",15.43,None,None,None,None,Triple Jump,None,None,...,15.1505,14.915,15.43,15.43,0.044,0.2795,0.515,-0.27,3.280255,Tier2
2,23153,"Kapil, Arnav",15.03m,WINGS ATHLETICS CLUB,21,1,Open,Triple Jump,0,,...,15.1505,14.915,15.03,15.03,-0.356,-0.1205,0.115,-0.67,0.732484,Tier4
3,912,"Kumar, Viresh",14.36m,Victoria Junior College,18,1,U20,Triple Jump,0,None,...,15.1505,14.915,14.36,14.36,-1.026,-0.7905,-0.555,-1.34,-3.535032,
4,5838,Tan Shou Yi Rei (Chen Shouyi),14.12,None,None,1,None,Triple Jump,None,None,...,15.1505,14.915,14.12,14.12,-1.266,-1.0305,-0.795,-1.58,-5.063694,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,3678,"{'90': 'CHEW, CAERISSE KAI YI', '1438': 'TNG, ...",12:22.0,Dunman High School,10,2,B Division,Relay,3200,None,...,NaN,NaN,742.0,742.00,NaN,NaN,NaN,NaN,NaN,
451,3639,"{'879': 'WONG YU XIM, ANDREA', '884': 'YONG, S...",01:54.3,Nanyang Girls' High School,10,2,C Division,Relay,800,None,...,NaN,NaN,114.3,114.30,NaN,NaN,NaN,NaN,NaN,
452,3654,"{'870': 'PUAR MIN, QUINN', '872': 'CHONG KATE ...",01:53.3,Nanyang Girls' High School,10,1,B Division,Relay,800,None,...,NaN,NaN,113.3,113.30,NaN,NaN,NaN,NaN,NaN,
453,3491,"{'813': 'LEA, PANG SUI ON', '814': 'CHARYNE, C...",37.74,CHIJ Primary (Toa Payoh),10,19,Junior 2,Relay,200,None,...,NaN,NaN,37.74,37.74,NaN,NaN,NaN,NaN,NaN,


In [831]:
top_eight_performers.to_csv('top_8_prod.csv', encoding='utf-8')

# Apply OCTC Rules

## OCTC Rule 4 - if athlete qualifies in 2 events, choose the better performing one

In [822]:
# Change to numeric

final_df[['2%', '3.5%', '5%', 'RESULT_CONV']] = final_df[['2%', '3.5%', '5%', 'RESULT_CONV']].apply(pd.to_numeric)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/4012151466.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['2%', '3.5%', '5%', 'RESULT_CONV']] = final_df[['2%', '3.5%', '5%', 'RESULT_CONV']].apply(pd.to_numeric)


In [823]:
# Select qualifiers based on 5% tolerance

#octc_rule3 = octc_rule4.loc[(((octc_rule4['CATEGORY_EVENT']=='Mid')|(octc_rule4['CATEGORY_EVENT']=='Sprint')|(octc_rule4['CATEGORY_EVENT']=='Long')|(octc_rule4['CATEGORY_EVENT']=='Hurdles')|(octc_rule4['CATEGORY_EVENT']=='Walk')|(octc_rule4['CATEGORY_EVENT']=='Relay')|(octc_rule4['CATEGORY_EVENT']=='Marathon')|(octc_rule4['CATEGORY_EVENT']=='Steeple')|(octc_rule4['CATEGORY_EVENT']=='Pentathlon')|(octc_rule4['CATEGORY_EVENT']=='Heptathlon')|(octc_rule4['CATEGORY_EVENT']=='Triathlon'))&(octc_rule4['RESULT_CONV'] <= octc_rule4['5%']) & (octc_rule4['age_extract']<40) & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon')))|(((octc_rule4['CATEGORY_EVENT']=='Jump')|(octc_rule4['CATEGORY_EVENT']=='Throw'))&(octc_rule4['RESULT_CONV'] >= octc_rule4['5%']) & (octc_rule4['age_extract']<40) & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon')))]

octc_rule4 = final_df.loc[
    (
    ((final_df['CATEGORY_EVENT']=='Mid')|(final_df['CATEGORY_EVENT']=='Sprint')|(final_df['CATEGORY_EVENT']=='Long')|(final_df['CATEGORY_EVENT']=='Hurdles')|(final_df['CATEGORY_EVENT']=='Walk')|(final_df['CATEGORY_EVENT']=='Relay')|(final_df['CATEGORY_EVENT']=='Marathon')|(final_df['CATEGORY_EVENT']=='Steeple')|(final_df['CATEGORY_EVENT']=='Pentathlon')|(final_df['CATEGORY_EVENT']=='Heptathlon')|(final_df['CATEGORY_EVENT']=='Triathlon'))
    
    & (final_df['RESULT_CONV'] <= final_df['5%']) 
    
#    & (octc_rule4['age_extract']<40) 
    
#    & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon'))
    )  
    |(
        ((final_df['CATEGORY_EVENT']=='Jump')|(final_df['CATEGORY_EVENT']=='Throw'))
        
    & (final_df['RESULT_CONV'] >= final_df['5%']) 
#    & (octc_rule4['age_extract']<40) 
#    & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon'))
    )
    ]

In [824]:
octc_rule4.to_csv('octc_rule4_prod.csv', encoding='utf-8')

In [825]:
#top_performers_final = top_performers_sorted.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[False, False, False], na_position='last')
octc_rule4_final = octc_rule4.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1)

In [826]:
# Rank list of athletes by group after sorting

#octc_rule3['Rank'] = (octc_rule3.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[False, False, True])['MAPPED_EVENT', 'GENDER']
#                .rank(method='first', ascending=False)
#             )


#octc_rule3 = octc_rule3.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'])
#octc_rule3['Rank'] = octc_rule3.groupby(['MAPPED_EVENT', 'GENDER']).cumcount() + 1

octc_rule4_final = octc_rule4_final.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
octc_rule4_final['Rank'] = octc_rule4_final.groupby(['MAPPED_EVENT', 'GENDER']).cumcount() + 1



In [827]:
octc_rule4_final

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank
234,23028,LEE GABRIEL JIN YI,15.67m,NUS,12,1,Open,Triple Jump,0,,...,0.284,0.51950,0.7550,-0.03,4.808917,Tier2,2003-02-23,2003.0,22.0,1
173,5676,"MEDINA, ANDREW GEORGE",15.43,None,None,None,None,Triple Jump,None,None,...,0.044,0.27950,0.5150,-0.27,3.280255,Tier2,NaT,NaN,NaN,2
248,23153,"Kapil, Arnav",15.03m,WINGS ATHLETICS CLUB,12,1,Open,Triple Jump,0,,...,-0.356,-0.12050,0.1150,-0.67,0.732484,Tier4,2003-08-25,2003.0,22.0,3
539,5725,"LOW, JUN YU",5.34,None,None,1,None,Pole Vault,None,None,...,0.244,0.32200,0.4000,0.14,7.692308,Tier 1,2001-04-21,2001.0,24.0,1
625,5655,SOH RUI YONG GUILLAUME,2:24:07,None,None,41,None,Marathon,None,None,...,888.980,1029.21500,1169.4500,702.00,12.508824,Tier 1,1991-08-06,1991.0,34.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10270,1695,"Ng, Bryan",10.95,Wings Athletics Club,12,5,Open,Dash,100,None,...,-0.291,-0.13425,0.0225,-0.50,0.215311,Tier4,2000-10-11,2000.0,25.0,22
10652,177,"Lee Shyen, Joshua",10.96,TeamFabian,17,1,U18,Dash,100,None,...,-0.301,-0.14425,0.0125,-0.51,0.119617,Tier4,2008-09-12,2008.0,17.0,23
10162,5614,"PEREIRA, VERONICA SHANTI",11.47,None,None,1,None,100m,None,None,...,0.515,0.69125,0.8675,0.28,7.382979,Tier 1,1996-09-20,1996.0,29.0,1
9881,23083,"TAN, ELIZABETH-ANN",11.99,SINGAPORE,12,1,Open,Dash,100,,...,-0.005,0.17125,0.3475,-0.24,2.957447,Tier3,2003-09-23,2003.0,22.0,2


In [828]:
octc_rule4_final.to_csv('octc_rule4_final.csv', encoding='utf-8')

## OCTC Rule 3 - Where top athlete is >30 yrs old (except marathon), to include next athlete as well (below 30)

In [257]:
# Apply Rule 3 & is within 5% performance band

octc_rule3 = octc_rule4_final.loc[(((octc_rule4['CATEGORY_EVENT']=='Mid')|(octc_rule4['CATEGORY_EVENT']=='Sprint')|(octc_rule4['CATEGORY_EVENT']=='Long')|(octc_rule4['CATEGORY_EVENT']=='Hurdles')|(octc_rule4['CATEGORY_EVENT']=='Walk')|(octc_rule4['CATEGORY_EVENT']=='Relay')|(octc_rule4['CATEGORY_EVENT']=='Marathon')|(octc_rule4['CATEGORY_EVENT']=='Steeple')|(octc_rule4['CATEGORY_EVENT']=='Pentathlon')|(octc_rule4['CATEGORY_EVENT']=='Heptathlon')|(octc_rule4['CATEGORY_EVENT']=='Triathlon'))&(octc_rule4['RESULT_CONV'] <= octc_rule4['5%']) & (octc_rule4['age_extract']<40) & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon')))|(((octc_rule4['CATEGORY_EVENT']=='Jump')|(octc_rule4['CATEGORY_EVENT']=='Throw'))&(octc_rule4['RESULT_CONV'] >= octc_rule4['5%']) & (octc_rule4['age_extract']<40) & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon')))]

In [258]:
octc_rule3

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract
2570,1607,nan,01:56.9,None,None,2,None,800m,None,None,...,116.90,-0.7220,0.98650,2.6950,-3.00,2.366111,Tier3,2006-06-24,2006.0,19.0
2580,1606,ZUBIN PERCY MUNCHERJI,01:53.7,None,None,2,None,800m,None,None,...,113.70,2.4780,4.18650,5.8950,0.20,5.175593,Tier 1,1996-06-23,1996.0,29.0
2433,20042,"YEE, CHUN WAI, ERIC",49.51m,Hwa Chong Alumni Association,12,1,Open,Discus Throw,0,(2kg),...,49.51,0.4904,1.24070,1.9910,-0.51,3.980408,Tier2,1999-03-01,1999.0,26.0
540,1600,Wei Xiang Gordon Lim,2:36:19,None,None,566,None,Marathon,None,None,...,9379.00,156.9800,297.21500,437.4500,-30.00,4.679110,Tier2,1993-02-07,1993.0,32.0
8604,22740,"Toh Jun Xi, Tedd",10.93,National University Singapore,12,4,Open,Dash,100,None,...,10.93,-0.2710,-0.11425,0.0425,-0.48,0.406699,Tier4,2002-03-18,2002.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10273,2816,Brayden Chan Wei Jie,00:10.86,RI,18.5,3,A,100m,None,None,...,10.86,-0.2010,-0.04425,0.1125,-0.41,1.076555,Tier4,NaT,NaN,18.5
3345,17663,"Aswin, Kumar",01:56.0,Temasek Polytechnic,12,4,Open,Run,800,,...,116.00,0.1780,1.88650,3.5950,-2.10,3.156277,Tier2,2006-04-13,2006.0,19.0
10347,18575,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,10.81,-0.1510,0.00575,0.1625,-0.36,1.555024,Tier3,2005-07-02,2005.0,20.0
8286,22810,"Adarsh, Aravinth",04:07.1,National University Singapore,12,2,Open,Run,1500,None,...,247.10,-2.9120,0.67900,4.2700,-7.70,1.783626,Tier3,2001-10-13,2001.0,24.0


In [259]:
# Rank list of athletes by group after sorting

#octc_rule3['Rank'] = (octc_rule3.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[False, False, True])['MAPPED_EVENT', 'GENDER']
#                .rank(method='first', ascending=False)
#             )


#octc_rule3 = octc_rule3.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'])
#octc_rule3['Rank'] = octc_rule3.groupby(['MAPPED_EVENT', 'GENDER']).cumcount() + 1

octc_rule3 = octc_rule3.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
octc_rule3['Rank'] = octc_rule3.groupby(['MAPPED_EVENT', 'GENDER']).cumcount() + 1



In [260]:
octc_rule3

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank
193,19066,LEE GABRIEL JIN YI,15.67m,NUS,12,1,Open,Triple Jump,0,,...,0.284,0.51950,0.7550,-0.03,4.808917,Tier2,2003-02-23,2003.0,22.0,1
206,19191,"Kapil, Arnav",15.03m,WINGS ATHLETICS CLUB,12,1,Open,Triple Jump,0,,...,-0.356,-0.12050,0.1150,-0.67,0.732484,Tier4,2003-08-25,2003.0,22.0,2
476,1763,"LOW, JUN YU",5.34,None,None,1,None,Pole Vault,None,None,...,0.244,0.32200,0.4000,0.14,7.692308,Tier 1,2001-04-21,2001.0,24.0,1
549,1693,SOH RUI YONG GUILLAUME,2:24:07,None,None,41,None,Marathon,None,None,...,888.980,1029.21500,1169.4500,702.00,12.508824,Tier 1,1991-08-06,1991.0,34.0,1
545,772,"TAN, AARON JUSTIN WEN JIE",2:35:26,INDIVIDUAL,31,None,None,Marathon,None,None,...,209.980,350.21500,490.4500,23.00,5.246016,Tier 1,1993-04-01,1993.0,32.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353,21106,MARK LEE REN,10.92,INDIVIDUAL,21,None,None,100m,None,None,...,-0.261,-0.10425,0.0525,-0.47,0.502392,Tier4,2004-04-01,2004.0,21.0,17
8604,22740,"Toh Jun Xi, Tedd",10.93,National University Singapore,12,4,Open,Dash,100,None,...,-0.271,-0.11425,0.0425,-0.48,0.406699,Tier4,2002-03-18,2002.0,23.0,18
9091,1652,"PEREIRA, VERONICA SHANTI",11.47,None,None,1,None,100m,None,None,...,0.515,0.69125,0.8675,0.28,7.382979,Tier 1,1996-09-20,1996.0,29.0,1
8826,19121,"TAN, ELIZABETH-ANN",11.99,SINGAPORE,12,1,Open,Dash,100,,...,-0.005,0.17125,0.3475,-0.24,2.957447,Tier3,2003-09-23,2003.0,22.0,2


In [261]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

octc_rule3.to_csv('octc_rule3.csv', encoding='utf-8')

## OCTC Rule 6 - max 6 for 100m/400m and max 3 for all other events

In [829]:
octc_rule6 = octc_rule4_final[(((octc_rule4_final['MAPPED_EVENT']=='400m')|(octc_rule4_final['MAPPED_EVENT']=='100m'))&(octc_rule4_final['Rank']<7))|(~((octc_rule4_final['MAPPED_EVENT']=='400m')|(octc_rule4_final['MAPPED_EVENT']=='100m'))&(octc_rule4_final['Rank']<4))]


In [830]:
octc_rule6.to_csv('octc_rule6.csv', sep=',', encoding='utf-8-sig', index=False)


In [364]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


# Download list of foreigners and spex carded athletes

In [365]:
#spex_list=pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/SPEX_CARDED_LIST.csv', encoding='latin-1')


In [1622]:
foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')


In [1623]:
#spex_list

In [432]:
foreigners

,LAST_NAME,FIRST_NAME,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Aaryan,Greuter Christoph,NaN,NaN,NaN
1,Akahodani,Takayuki,NaN,NaN,NaN
2,Apondar,Audric,NaN,NaN,NaN
3,Brooks,Ruby,NaN,NaN,NaN
4,Brouwer,Cees,NaN,NaN,NaN
...,...,...,...,...,...
219,CHO,CHIA-HSUAN,NaN,NaN,NaN
220,NGUYEN,HOAI VAN,NaN,NaN,NaN
221,NGUYEN,QUOC THINH,NaN,NaN,NaN
222,PHAM,QUYNH GIANG,NaN,NaN,NaN


In [433]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

In [434]:
foreigners

,LAST_NAME,FIRST_NAME,Unnamed: 2,Unnamed: 3,Unnamed: 4,V1,V2,V3,V4
0,Aaryan,Greuter Christoph,NaN,NaN,NaN,Aaryan Greuter Christoph,Greuter Christoph Aaryan,"Aaryan, Greuter Christoph",Greuter Christoph Aaryan
1,Akahodani,Takayuki,NaN,NaN,NaN,Akahodani Takayuki,Takayuki Akahodani,"Akahodani, Takayuki",Takayuki Akahodani
2,Apondar,Audric,NaN,NaN,NaN,Apondar Audric,Audric Apondar,"Apondar, Audric",Audric Apondar
3,Brooks,Ruby,NaN,NaN,NaN,Brooks Ruby,Ruby Brooks,"Brooks , Ruby",Ruby Brooks
4,Brouwer,Cees,NaN,NaN,NaN,Brouwer Cees,Cees Brouwer,"Brouwer, Cees",Cees Brouwer
...,...,...,...,...,...,...,...,...,...
219,CHO,CHIA-HSUAN,NaN,NaN,NaN,CHO CHIA-HSUAN,CHIA-HSUAN CHO,"CHO, CHIA-HSUAN",CHIA-HSUAN CHO
220,NGUYEN,HOAI VAN,NaN,NaN,NaN,NGUYEN HOAI VAN,HOAI VAN NGUYEN,"NGUYEN , HOAI VAN",HOAI VAN NGUYEN
221,NGUYEN,QUOC THINH,NaN,NaN,NaN,NGUYEN QUOC THINH,QUOC THINH NGUYEN,"NGUYEN , QUOC THINH",QUOC THINH NGUYEN
222,PHAM,QUYNH GIANG,NaN,NaN,NaN,PHAM QUYNH GIANG,QUYNH GIANG PHAM,"PHAM , QUYNH GIANG",QUYNH GIANG PHAM


In [371]:
#spex1 = spex_list['V1'].dropna().tolist()
#spex2 = spex_list['V2'].dropna().tolist()
#spex3 = spex_list['V3'].dropna().tolist()
#spex4 = spex_list['V4'].dropna().tolist()
#spex5 = spex_list['V5'].dropna().tolist()


In [435]:
for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()


In [373]:
#spex_athletes=spex1+spex2+spex3+spex4+spex5

In [436]:
foreign_list = for1+for2+for3+for4 

In [437]:
#spex_athletes

In [438]:
foreign_list

['Aaryan Greuter Christoph',
 'Akahodani Takayuki',
 'Apondar Audric',
 'Brooks  Ruby',
 'Brouwer Cees',
 'CARLESATER ARMAAN',
 'Chary Mathangi ',
 'Cinthikael Angel',
 'Comia Tsang Hannah',
 'Comia Tsang Mhandy',
 'Donnelly Jordan ',
 'Dorai Trinity Sharilyn',
 'Freeman Alexander',
 'Gaume Constance',
 'Gregorio John Alexander',
 'HAGIWARA RYOTA ',
 'Indarto Gerard',
 'Joy Matthew',
 'JULURI ADITYA',
 'Kiet Tran',
 'Koduru Kalyani',
 'Kuchenbuch Natalya',
 'KUIJPERS KUIJPERS AYKO ELLIN',
 'Kumar Joseph Suraj',
 'KUPPUSAMY PRAKSHEETA',
 'Kusumo Naia',
 'Kuwalekar  Spruha ',
 'Lahey Kezia Stephanie ',
 'Liz Jerry',
 'Marican Raees Marican Bin Ayoob Marican',
 'MARIE BRUNO DE FERRIERES DE SAUVEBOEUF LOUIS ',
 'Maslov Timofei',
 'Masrezwan Maeva',
 'MENDOZA NICCOLO',
 'Mennella Leonardo',
 'Meyers Nikita Mae Jiny-Yu',
 'Mishra Sunay',
 'Mo Thant Cin',
 'Moon Daniel Youngjo',
 'OCAMPO ANIAG DWAYNE MICHAEL OCAMPO ANIAG',
 'Ortega Rafael Pedro',
 'Owen Elia',
 'owl emma',
 'Ozbudak Deniz',
 

In [439]:
#spex_athletes_casefold=[s.casefold() for s in spex_athletes]

In [440]:
foreign_list_casefold=[s.casefold() for s in foreign_list]

In [442]:
#spex_athletes_casefold

In [443]:
foreign_list_casefold

['aaryan greuter christoph',
 'akahodani takayuki',
 'apondar audric',
 'brooks  ruby',
 'brouwer cees',
 'carlesater armaan',
 'chary mathangi ',
 'cinthikael angel',
 'comia tsang hannah',
 'comia tsang mhandy',
 'donnelly jordan ',
 'dorai trinity sharilyn',
 'freeman alexander',
 'gaume constance',
 'gregorio john alexander',
 'hagiwara ryota ',
 'indarto gerard',
 'joy matthew',
 'juluri aditya',
 'kiet tran',
 'koduru kalyani',
 'kuchenbuch natalya',
 'kuijpers kuijpers ayko ellin',
 'kumar joseph suraj',
 'kuppusamy praksheeta',
 'kusumo naia',
 'kuwalekar  spruha ',
 'lahey kezia stephanie ',
 'liz jerry',
 'marican raees marican bin ayoob marican',
 'marie bruno de ferrieres de sauveboeuf louis ',
 'maslov timofei',
 'masrezwan maeva',
 'mendoza niccolo',
 'mennella leonardo',
 'meyers nikita mae jiny-yu',
 'mishra sunay',
 'mo thant cin',
 'moon daniel youngjo',
 'ocampo aniag dwayne michael ocampo aniag',
 'ortega rafael pedro',
 'owen elia',
 'owl emma',
 'ozbudak deniz',
 

In [444]:
# Exclusion list is foreigners + spex carded 

#exclusions = foreign_list_casefold + spex_athletes_casefold

exclusions = foreign_list_casefold

In [445]:
# Apply Rule 4 above...if an athlete is in more than one event, choose the best performing one

top_performers=rslt_df.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [446]:
top_performers

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR
8353,11670,{},12.99,8.0,2,Boys 4x100 Meter Relay 15-16,2015-01-01 00:00:00.000,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,12.99,27.1572,27.74760,28.3380,71.996951
12480,17949,"{9: 'Ng, Caitlin Shan Wen', 438: 'Seow, Kyra',...",22.17,0.0,1,Girls 13-14 4x100 Meter Relay U15,None,None,Relay,Female,...,4 x 100m relay,44.58,45.4716,46.14030,46.8090,22.17,23.3016,23.97030,24.6390,55.269179
9267,13141,"{995: 'Lee, Asher', 991: 'Yeo, Chee Hean Phili...",29.89,0.0,2,Boys 9-12 4x400 Meter Relay U13,None,None,Relay,Male,...,4 x 400m relay,188.82,192.5964,195.42870,198.2610,29.89,162.7064,165.53870,168.3710,89.170109
9588,13486,"{994: 'De Ming, Gilbert Lee', 988: 'Lee, Jerem...",38.89,0.0,14,Boys 10-12 4x100 Meter Relay U13,None,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,38.89,1.2572,1.84760,2.4380,6.194106
12424,17876,"{991: 'Yeo, Chee Hean Philip', 992: 'Yeo, Chee...",12.92,0.0,16,Boys 10-12 4x100 Meter Relay U13,None,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,12.92,27.2272,27.81760,28.4080,72.174797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10975,15886,"., Khairulnazim",14.72,17.0,4,Men 400 Meter Dash Open,2006-01-27,None,Sprint,Male,...,400m,46.63,47.5626,48.26205,48.9615,14.72,32.8426,33.54205,34.2415,73.432340
9918,13873,"., Kaarthika",12.53,27.0,9,Women 400 Meter Dash Women Junior,1996-06-13,None,Sprint,Female,...,400m,53.84,54.9168,55.72440,56.5320,12.53,42.3868,43.19440,44.0020,81.727340
10701,15018,"., Jayashree",1:14.05,15.0,36,Girls Long Jump B Div,2008-01-29 00:00:00.000,None,Jump,Female,...,Long jump,6.02,5.8996,5.80930,5.7190,10.97,5.0704,5.16070,5.2510,87.225914
12077,17484,"., Cheung Zheng",47.61,14.0,20,Boys 11-14 400 Meter Dash U15,2009-02-19,None,Sprint,Male,...,400m,46.63,47.5626,48.26205,48.9615,47.61,-0.0474,0.65205,1.3515,2.898349


In [447]:
top_performers.to_csv('top_checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)

In [448]:
excluded_list = top_performers.loc[~rslt_df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [449]:
#spexed_list=top_performers

In [450]:
excluded_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
excluded_list['overall_rank'] = 1
excluded_list['overall_rank'] = excluded_list.groupby(['MAPPED_EVENT', 'GENDER'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_list['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [451]:
#spexed_list=spexed_list[(((spexed_list['EVENT']=='400m')&(spexed_list['overall_rank']<7)))|(((spexed_list['EVENT']=='100m')&(spexed_list['overall_rank']<7)))]

In [452]:
#Apply OCTC selection rule: max 6 for 100m/400m and max 3 for all other events

excluded_list = excluded_list[(((excluded_list['MAPPED_EVENT']=='400m')|(excluded_list['MAPPED_EVENT']=='100m'))&(excluded_list['overall_rank']<7))|(~((excluded_list['MAPPED_EVENT']=='400m')|(excluded_list['MAPPED_EVENT']=='100m'))&(excluded_list['overall_rank']<4))]


In [453]:
excluded_list

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR,overall_rank
8135,11374,"Tan, Bernice",2:16.97,23.0,3,Women 10000 Meter Run Open,2000-11-16,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,1
8137,11380,"NicAmhlaoibh, Maire",2:16.97,35.0,2,Women 10000 Meter Run Open,1988-02-14,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,2
8134,11372,"Ng, Yew Cheo",2:16.97,37.0,4,Women 10000 Meter Run Open,1986-11-30,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,3
9739,13637,"PHAGAMI, KHUM BAHADUR",20.34m,39.0,2,Men 10000 Meter Run Men Senior A,1984-08-09,None,Long,Male,...,1885.55,1923.2610,1951.54425,1979.8275,144.29,1778.9710,1807.25425,1835.5375,97.347591,1
9734,13632,"Mohd, Jamian",35.31m,38.0,6,Men 10000 Meter Run Men Senior A,1985-04-29,None,Long,Male,...,1885.55,1923.2610,1951.54425,1979.8275,144.29,1778.9710,1807.25425,1835.5375,97.347591,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3249,3544,SOLANA LEANN REINKIRSTEN CANDA,00:22.7,0.0,9.0,Triple Jump,None,None,Jump,Female,...,13.46,13.1908,12.98890,12.7870,1150.60,1137.4092,1137.61110,1137.8130,8453.291233,2
3260,3555,SHERYL TOH PEIXUAN,00:23.4,0.0,6.0,Triple Jump,None,None,Jump,Female,...,13.46,13.1908,12.98890,12.7870,1150.60,1137.4092,1137.61110,1137.8130,8453.291233,3
3257,3552,RAO TIANYU,00:28.7,0.0,12.0,Triple Jump,None,None,Jump,Male,...,15.70,15.3860,15.15050,14.9150,1150.60,1135.2140,1135.44950,1135.6850,7233.662420,1
3246,3541,NG JUN JIE,00:15.7,0.0,11.0,Triple Jump,None,None,Jump,Male,...,15.70,15.3860,15.15050,14.9150,1150.60,1135.2140,1135.44950,1135.6850,7233.662420,2


In [454]:
excluded_list.to_csv('octc_all_events.csv', sep=',', encoding='utf-8-sig', index=False)


In [ ]:
# 1. If top athlete>30 (and the only one >30), there are already 6 in list, and next one is already <30 then it means do nothing? 
# 2. If top athlete>30 and is the sole pick.  Next athlete is far beyond 5% band.  Do we add?
# 3. If top athlete>30 and there are less than 6 within 5% band of SEAG benchmark. Do we add that one more who is beyond 5%?
# 4. At least one per gender.  The only pick is far beyond 5%.  Do we add?

In [ ]:
# Convert time format for marathon and 5000m into mm:ss.00
import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(top_performers_clean)):
        
    rowIndex = top_performers_clean.index[i]

    event=top_performers_clean.iloc[rowIndex,19]
        
    
    time_base2=top_performers_clean.iloc[rowIndex,24]
    time_base3=top_performers_clean.iloc[rowIndex,25]
    time_base5=top_performers_clean.iloc[rowIndex,26]
    
        
    if metric==None:
        continue
        
    if event=='800m' or event=='10000m' or event=='5000m' or event=='3000m steeplechase' or event=='1500m':
        
        print(time_base2, time_base3, time_base5)

        
        
        try:
            
        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            output2 = datetime.datetime.strftime(date_preconvert2, "%M:%S.%f")
            output3 = datetime.datetime.strftime(date_preconvert3, "%M:%S.%f")
            output5 = datetime.datetime.strftime(date_preconvert5, "%M:%S.%f")
            
       #     print(output2, output3, output5)

                    
       #     top_performers_clean.loc[rowIndex, '2%_timing'] = output2
       #     top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
       #     top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
   
            top_performers_clean.at[rowIndex, '2%'] = output2 # copy over time format
            top_performers_clean.at[rowIndex, '3.5%'] = output3
            top_performers_clean.at[rowIndex, '5%'] = output5


            
        except:
            
            pass
        

        
    elif event=='Marathon':
        
        print(time_base2, time_base3, time_base5)

        
        try:
            

        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            
            
            output2 = datetime.datetime.strftime(date_preconvert2, "%H:%M:%S")
            output3 = datetime.datetime.strftime(date_preconvert3, "%H:%M:%S")
            output5 = datetime.datetime.strftime(date_preconvert5, "%H:%M:%S")

            
        
        #    top_performers_clean.loc[rowIndex, '2%_timing'] = output2
        #    top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
        #    top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
            top_performers_clean.at[rowIndex, '2%'] = output2 # copy over time format
            top_performers_clean.at[rowIndex, '3.5%'] = output3
            top_performers_clean.at[rowIndex, '5%'] = output5

            
         #   print('output', output2, output3, output5)


        
        except:
            
            pass
                        
             


# Marinda's request for 6 and 8 place finish analysis

In [4062]:
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND (RANK='3' OR RANK='6' OR RANK='8')
"""

SEAG = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4064]:
SEAG.tail(60)

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
53,Farell Glen Felix Jerus,2.15 m,3,High jump,Jump,Male,SEA Games,Final
54,Sun Soklim,NM,6,High jump,Jump,Female,SEA Games,Final
55,Kobsit Sittichai,2.07 m,6,High jump,Jump,Male,SEA Games,Final
56,Suwandi Wijaya,7.47,6,Long jump,Jump,Male,SEA Games,Final
57,Pok Pisey,4.76,8,Long jump,Jump,Female,SEA Games,Final
58,Sapwaturrahman Sapwaturrahman,7.62,3,Long jump,Jump,Male,SEA Games,Final
59,Bùi Thị Loan,6.02,3,Long jump,Jump,Female,SEA Games,Final
60,Nurul Ashikin Abas,5.67,6,Long jump,Jump,Female,SEA Games,Final
61,John Marvin Aragon Rafols,7.19,8,Long jump,Jump,Male,SEA Games,Final
62,Sunisa Khotseemueang,None,3,Heptathlon,Heptathlon,Female,SEA Games,Final


In [247]:
for i in range(len(SEAG)):
        
    rowIndex = SEAG.index[i]

    input_string=SEAG.iloc[rowIndex,3]  # event is column index 3
    
    metric=SEAG.iloc[rowIndex,1] # result is column index 1
    
    if (metric==None or metric=='NH' or metric=='NM'):  # skip those non-numeric values
        continue
        
   # elif metric=='NH':
    #    continue
        
    print(i, input_string, metric)
        
    out = convert_time(i, input_string, metric)
     
    SEAG.loc[rowIndex, 'Metric'] = out

0 1500m     03:59.40
1 1500m     04:04.3
2 1500m     04:05.4
3 Decathlon 6891
4 100m 10.78
5 100m 11.75
6 100m 11.96
7 100m 10.443
8 200m 24.09
9 200m 23.6
11 200m 21.02
12 200m 21.58
13 400m 46.63
14 400m 48.26
15 400m 57.54
16 400m 1:01.20
17 400m 53.84
18 800m 1:57.98
19 800m 1:53.86
20 800m 2:15.77
21 800m 2:18.25
22 800m 2:09.15
23 800m 1:55.39
24 1500m 4:26.33
25 1500m 4:38.64
26 1500m 5:00.090
27 5000m 17:13.63
28 5000m 18:41.76
29 5000m 14:43.45
30 5000m 15:01.79
31 5000m 18:02.52
32 5000m 15:09.65
33 10000m 33:00.23
34 10000m 32:36.32
35 10000m 38:11.04
36 10000m 37:29.58
37 10000m 35:31.03
38 10000m 31:25.55
39 Marathon 2:35:49
40 Marathon 3:17:36
41 Marathon 3:46:44
42 Marathon 2:52:41
43 Marathon 2:50:27
44 Marathon 2:41:36
45 Shot put 11.69
46 Shot put 12.12
47 Shot put 14.44
48 Shot put 14.65
49 Shot put 17.3
50 Shot put 16.06
52 High jump 2.07 m
53 High jump 2.15 m
54 High jump 1.73
55 High jump 1.95 m
56 Long jump 7.47
57 Long jump 5.67
58 Long jump 6.02
59 Long jump 4.

In [248]:
SEAG

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78
...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06


In [249]:
SEAG.to_csv('check_variation.csv', encoding='utf-8')

In [250]:
comps = SEAG[SEAG['RANK']=='3']

In [251]:
comps

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.400
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.000
5,Trần Thị Nhi Yến,11.75,3,100m,Sprint,Female,SEA Games,Final,11.750
7,Muhammad Haiqal Hanafi,10.443,3,100m,Sprint,Male,SEA Games,Final,10.443
9,Zaidatul Husniah Zulkifli,23.6,3,200m,Sprint,Female,SEA Games,Final,23.600
11,Lalu Muhammad Zohri,21.02,3,200m,Sprint,Male,SEA Games,Final,21.020
13,Frederick Ramirez,46.63,3,400m,Sprint,Male,SEA Games,Final,46.630
17,Nguyễn Thị Hằng,53.84,3,400m,Sprint,Female,SEA Games,Final,53.840
19,Wan Muhammad Fazri Wan Zahari,1:53.86,3,800m,Mid,Male,SEA Games,Final,113.860
22,Goh Chui Ling,2:09.15,3,800m,Mid,Female,SEA Games,Final,129.150


In [252]:
new_comps = comps[['EVENT', 'GENDER', 'Metric']]

In [253]:
#comps=SEAG.groupby(['EVENT', 'GENDER'])['Metric'].min()


In [254]:
df2 = SEAG.merge(new_comps, on=['EVENT', 'GENDER'], how='left')


In [255]:
df2

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric_x,Metric_y
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40,239.400
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30,239.400
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40,239.400
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00,6891.000
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78,10.443
...,...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29,219.290
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85,660.850
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05,535.050
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06,535.050


In [256]:
df2.to_csv('check_metric.csv', encoding='utf-8')

In [257]:
df2['% VARIATION']=(df2['Metric_x']/df2['Metric_y']*100-100)

In [258]:
df2

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric_x,Metric_y,% VARIATION
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40,239.400,0.000000
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30,239.400,2.046784
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40,239.400,2.506266
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00,6891.000,0.000000
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78,10.443,3.227042
...,...,...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29,219.290,0.000000
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85,660.850,0.000000
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05,535.050,0.000000
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06,535.050,18.878609


In [259]:
df2.to_csv('SEAG_variation.csv', sep=',', encoding='utf-8-sig', index=False)


# Convert timing format from seconds to HMSf

In [4803]:
datetime.datetime.fromtimestamp(120).strftime('%M:%S.%f')
        

'32:00.000000'

In [4808]:
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d:%02d:%02d" % (hour, minutes, seconds)

In [5295]:
import datetime
def format_result(result):
    date = datetime.datetime.utcfromtimestamp(result)
    output = datetime.datetime.strftime(date, "%H:%M:%S.%f")
    return output
print(format_result(113.86))

00:01:53.860000


# Clean manual load data

In [4]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/SEA Games/')


df_SHA = pd.read_csv("SEA_manual_additions.csv", encoding='latin-1')

In [5]:
df_SHA

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,NaN,NaN,NaN,Agus Prayogo,1.0,NaN,NaN,NaN,2:32:59,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
1,NaN,NaN,NaN,ÊArlan Estobo Arbois,2.0,NaN,NaN,NaN,2:33:27,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
2,NaN,NaN,NaN,Nguyen Thanh Hoang,3.0,NaN,NaN,NaN,2:35:49,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
3,NaN,NaN,NaN,Tan Huong Leong,4.0,NaN,NaN,NaN,2:40:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
4,NaN,NaN,NaN,Vanh Pheara,5.0,NaN,NaN,NaN,2:41:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
5,NaN,NaN,NaN,Quoc Luong Trinh,6.0,NaN,NaN,NaN,2:41:36,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
6,NaN,NaN,NaN,Sanchai Namkhet,7.0,NaN,NaN,NaN,2:43:47,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
7,NaN,NaN,NaN,Joanito Fernandes,8.0,NaN,NaN,NaN,2:52:41,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
8,NaN,NaN,NaN,Tony Ah-Thit Payne,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
9,NaN,NaN,NaN,Yang Piseth,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN


In [6]:
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\xa0', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\r', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\n', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.strip()


In [7]:
df_SHA.to_csv('SEA_manual_additions.csv', index=False, encoding='utf-8')

# Read a csv of name list variations

In [2566]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

names = pd.read_csv("name_variations.csv")

In [2567]:
names

,VARIATION,NAME
0,^Harry\sIrfan\sCurra$,HARRY IRFAN CURRAN
1,"Curran, Harry Irfann",HARRY IRFAN CURRAN
2,"^Curran,\sHarry$",HARRY IRFAN CURRAN
3,Harry Irfan Curran Q,HARRY IRFAN CURRAN
4,Wei Jun Huang,HUANG WEI JUN
...,...,...
182,Chua Hsin-Wen Clara,"Chua Hsin-Wen, Clara"
183,Chua Hsin-Wen Clara Q,"Chua Hsin-Wen, Clara"
184,"HSIN-WEN CLARA, Chua","Chua Hsin-Wen, Clara"
185,CHUA HSIN-WEN CLARA,"Chua Hsin-Wen, Clara"


In [ ]:
lines = [line.strip() for line in open('file.csv')]
for x in lines:
    match=re.search(r'^_.*_$',x)
    if match: print x

In [ ]:
df['NAME'] = df['NAME'].replace(regex=r'^Harry\sIrfan\sCurra$', value='HARRY IRFAN CURRAN')


In [2568]:
for index, row in names.iterrows():
    print(row['VARIATION'])
    

^Harry\sIrfan\sCurra$
Curran, Harry Irfann
^Curran,\sHarry$
Harry Irfan Curran Q
Wei Jun Huang
HUANG, WEI JUN
., Hariharan
S/O KrishnS/O Krishna, Hariharan
KRISHNAN, HARIHARAN S/O
Andrew George Medina
Medina, Andrew
Medina, Andrew George
George Medina, Andrew
George George Medina, Andrew
Gabriel Lee
Lee, Gabriel
Jun Yu Low
Low Jun Yu
Low, Jun Yu
Caleb Hia
Melvin, Wong
MELVIN, WONG
Conrad Kangli Emery
Conrad, Kangli Emery
Kangli Emery Conrad
Conrad Kangli Emery Q
Mun Jern Wei
Mun, Jern Wei Ivan
Mun, Ivan
Wong Yijie Lucas
Wong, Yijie Lucas
Tung Hon Andrew Pak
Pak, Andrew
Brandon Heng Fu Hong
Heng, Brandon
Brandon Heng Fu Hong
HENG, BRANDON
HENG, FU HONG, BRANDON
HENG, FU HONG BRANDON
Mohamed, Haja Fayiz
Osman, Amir Rusyaidi
nan
Goh, Shaun
Shaun Goh
Ethan Yan
TAN, AARON JUSTIN
Tan, Aaron Justin
Rui Yong Soh
YEE, ERIC CHUN WAI
Yee, Eric
YEE CHUN WAI, ERIC
Eric Yee Chun Wai
Yee Chun Wai, Eric
Yee, Chun Wai, Eric
Yee Chun Wai, Eric
YEE, ERIC CHUN WAI
Lim, Oliver
Jie Cong Jayden, Tan
TAN , JI